tackling dummies need not apply

In [308]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

pd.options.display.max_columns = None

import math

In [2]:
data = {
  'week': [1,1,1,1,1,1,1,1,1],
  'day': [1,1,1,2,2,2,3,3,3],
    'hour': [1,2,3,1,2,3,1,2,3],
    'value': [0,123,0,456,0,0,0,789,0]
}
simple = pd.DataFrame(data)


In [3]:
simple

,week,day,hour,value
0,1,1,1,0
1,1,1,2,123
2,1,1,3,0
3,1,2,1,456
4,1,2,2,0
5,1,2,3,0
6,1,3,1,0
7,1,3,2,789
8,1,3,3,0


In [4]:
simple['value'] = simple.groupby(['week', 'day'])['value'].transform(max)
simple

,week,day,hour,value
0,1,1,1,123
1,1,1,2,123
2,1,1,3,123
3,1,2,1,456
4,1,2,2,456
5,1,2,3,456
6,1,3,1,789
7,1,3,2,789
8,1,3,3,789


TO DO:

Create tackle offensive and defensive influencers; possibly anyone within half-yard of tackle point?

Confirm all tacklers are at point of tackle with distance at zero

Angle from ball carrier = ballVector; need oBall as well for ball orientation vs the tackler orientation

Use just the x distance for the distances the ball travels at each event; key event is minimize transfer to tackle yards

Create one row with all tacklers, ball carrier, influencers (any others?) for machine learning predictions


Before merging all the weeks into events and then an overall events dataframe:

List all pre-cleaning needed for each week before isolating events; what frames without events need to be used in any data modifications? NO NONE EVENTS FRAMES NEED TO BE USED

Pre-cleaning might be ensure all rows from tackles dataframe are part of tackles events in larger file. For example: are all missed tackles included? NOT NECESSARY

momentum is speed times mass; force is acceleration times mass

In [5]:
week1 = pd.read_csv('./data/tracking_week_1.csv')

In [6]:
games = pd.read_csv('./data/games.csv')
players = pd.read_csv('./data/players.csv')
plays = pd.read_csv('./data/plays.csv')
tackles = pd.read_csv('./data/tackles.csv')
#week1 = pd.read_csv('./data/tracking_week_1.csv')
week2 = pd.read_csv('./data/tracking_week_2.csv')
week3 = pd.read_csv('./data/tracking_week_3.csv')
week4 = pd.read_csv('./data/tracking_week_4.csv')
week5 = pd.read_csv('./data/tracking_week_5.csv')
week6 = pd.read_csv('./data/tracking_week_6.csv')
week7 = pd.read_csv('./data/tracking_week_7.csv')
week8 = pd.read_csv('./data/tracking_week_8.csv')
week9 = pd.read_csv('./data/tracking_week_9.csv')

In [7]:
'''week1_events = week1[week1['event'].notna()]
week2_events = week2[week2['event'].notna()]
week3_events = week3[week3['event'].notna()]
week4_events = week4[week4['event'].notna()]
week5_events = week5[week5['event'].notna()]
week6_events = week6[week6['event'].notna()]
week7_events = week7[week7['event'].notna()]
week8_events = week8[week8['event'].notna()]
week9_events = week9[week9['event'].notna()]'''

"week1_events = week1[week1['event'].notna()]\nweek2_events = week2[week2['event'].notna()]\nweek3_events = week3[week3['event'].notna()]\nweek4_events = week4[week4['event'].notna()]\nweek5_events = week5[week5['event'].notna()]\nweek6_events = week6[week6['event'].notna()]\nweek7_events = week7[week7['event'].notna()]\nweek8_events = week8[week8['event'].notna()]\nweek9_events = week9[week9['event'].notna()]"

In [8]:
#week_events = [week1_events, week2_events, week3_events, week4_events, week5_events, week6_events, 
#               week7_events, week8_events, week9_events]

In [9]:
#all_events = pd.concat(week_events)

In [10]:
games

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore
0,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
1,2022091100,2022,1,09/11/2022,13:00:00,ATL,NO,26,27
2,2022091101,2022,1,09/11/2022,13:00:00,CAR,CLE,24,26
3,2022091102,2022,1,09/11/2022,13:00:00,CHI,SF,19,10
4,2022091103,2022,1,09/11/2022,13:00:00,CIN,PIT,20,23
...,...,...,...,...,...,...,...,...,...
131,2022110607,2022,9,11/06/2022,13:00:00,WAS,MIN,17,20
132,2022110608,2022,9,11/06/2022,16:05:00,ARI,SEA,21,31
133,2022110609,2022,9,11/06/2022,16:25:00,TB,LA,16,13
134,2022110610,2022,9,11/06/2022,20:20:00,KC,TEN,20,17


In [11]:
games_weeks = games[['gameId', 'week']]

In [12]:
games_weeks

,gameId,week
0,2022090800,1
1,2022091100,1
2,2022091101,1
3,2022091102,1
4,2022091103,1
...,...,...
131,2022110607,9
132,2022110608,9
133,2022110609,9
134,2022110610,9


In [13]:
players

,nflId,height,weight,birthDate,collegeName,position,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan
...,...,...,...,...,...,...,...
1678,55200,6-6,266,NaN,Indiana,DT,Ryder Anderson
1679,55212,6-0,230,NaN,Iowa State,ILB,Jake Hummel
1680,55239,6-2,300,NaN,Pennsylvania,DT,Prince Emili
1681,55240,6-1,185,NaN,Buffalo,CB,Ja'Marcus Ingram


In [14]:
players['totalInches'] = (players.height.str.split("-").str[0].astype(int) * 12) + (
    players.height.str.split("-").str[1].astype(int)
)
players['mass'] = (players['totalInches']/12 * players['weight']).round(2)

In [15]:
plays

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,...,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,...,0.160485,0.839515,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,...,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,...,0.620552,0.379448,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,...,0.836290,0.163710,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12481,2022100204,123,43293,Ezekiel Elliott,(13:31) E.Elliott right tackle to WAS 38 for 1...,1,1,10,DAL,WAS,...,0.638600,0.361400,-0.011542,0.011542,3.642571,-0.504018,NaN,NaN,NaN,NaN
12482,2022091200,3467,46189,Will Dissly,(6:08) G.Smith pass short right to W.Dissly to...,4,1,10,SEA,DEN,...,0.615241,0.384759,-0.025458,0.025458,1.434580,-0.444642,NaN,NaN,NaN,NaN
12483,2022101605,3371,44860,Joe Mixon,(9:35) (Shotgun) J.Mixon left end to CIN 47 fo...,4,1,10,CIN,NO,...,0.667054,0.332946,-0.005164,0.005164,2.115356,0.203819,NaN,NaN,NaN,NaN
12484,2022100207,2777,52449,Jonathan Taylor,(2:02) (Shotgun) J.Taylor up the middle to TEN...,3,1,10,IND,TEN,...,0.410611,0.589389,-0.046648,0.046648,3.946232,-0.976039,NaN,NaN,NaN,NaN


In [16]:
print(plays.columns.tolist())

['gameId', 'playId', 'ballCarrierId', 'ballCarrierDisplayName', 'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore', 'passResult', 'passLength', 'penaltyYards', 'prePenaltyPlayResult', 'playResult', 'playNullifiedByPenalty', 'absoluteYardlineNumber', 'offenseFormation', 'defendersInTheBox', 'passProbability', 'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability', 'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded', 'expectedPoints', 'expectedPointsAdded', 'foulName1', 'foulName2', 'foulNFLId1', 'foulNFLId2']


In [17]:
plays['passResult'].value_counts()

C    5634
R     471
Name: passResult, dtype: int64

In [18]:
np.count_nonzero(plays['passResult'])

12486

In [19]:
plays['passResult'].nunique()

2

In [20]:
plays['gamePlayId'] = plays[['gameId', 'playId']].astype(str).apply(lambda x: ''.join(x), axis=1)
plays['gamePlayId'] = plays['gamePlayId'].astype('int64')

In [21]:
plays = pd.merge(plays, games_weeks, on='gameId', how='left')

In [22]:
plays

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2,gamePlayId,week
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,...,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN,20221009083537,5
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,...,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN,20220911033126,1
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,...,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN,20220911111148,1
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,...,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN,20221002122007,4
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,...,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN,20220919001372,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12481,2022100204,123,43293,Ezekiel Elliott,(13:31) E.Elliott right tackle to WAS 38 for 1...,1,1,10,DAL,WAS,...,-0.011542,0.011542,3.642571,-0.504018,NaN,NaN,NaN,NaN,2022100204123,4
12482,2022091200,3467,46189,Will Dissly,(6:08) G.Smith pass short right to W.Dissly to...,4,1,10,SEA,DEN,...,-0.025458,0.025458,1.434580,-0.444642,NaN,NaN,NaN,NaN,20220912003467,1
12483,2022101605,3371,44860,Joe Mixon,(9:35) (Shotgun) J.Mixon left end to CIN 47 fo...,4,1,10,CIN,NO,...,-0.005164,0.005164,2.115356,0.203819,NaN,NaN,NaN,NaN,20221016053371,6
12484,2022100207,2777,52449,Jonathan Taylor,(2:02) (Shotgun) J.Taylor up the middle to TEN...,3,1,10,IND,TEN,...,-0.046648,0.046648,3.946232,-0.976039,NaN,NaN,NaN,NaN,20221002072777,4


In [23]:
plays['week'].value_counts()

5    1495
3    1474
1    1473
4    1435
2    1415
8    1412
7    1317
6    1286
9    1179
Name: week, dtype: int64

In [24]:
print(plays.columns.tolist())

['gameId', 'playId', 'ballCarrierId', 'ballCarrierDisplayName', 'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam', 'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock', 'preSnapHomeScore', 'preSnapVisitorScore', 'passResult', 'passLength', 'penaltyYards', 'prePenaltyPlayResult', 'playResult', 'playNullifiedByPenalty', 'absoluteYardlineNumber', 'offenseFormation', 'defendersInTheBox', 'passProbability', 'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability', 'homeTeamWinProbabilityAdded', 'visitorTeamWinProbilityAdded', 'expectedPoints', 'expectedPointsAdded', 'foulName1', 'foulName2', 'foulNFLId1', 'foulNFLId2', 'gamePlayId', 'week']


In [25]:
plays['offenseFormation'].value_counts()

SHOTGUN       6378
SINGLEBACK    3524
I_FORM         929
EMPTY          850
PISTOL         599
JUMBO          115
WILDCAT         87
Name: offenseFormation, dtype: int64

In [26]:
plays['playResult'].unique()

array([  9,   3,  15,   7,   5,  11,   6,   0,  10,   8,   2,  -1,  -4,
         1,  -3,  48,  31,  -2,  23,  18,  22,  26,  21,  41,  17,  24,
        19,   4,  25,  29,  12, -10,  14, -19,  -7,  20,  13,  -5,  32,
        16,  34,  50,  30,  -6,  45,  52,  -8,  38,  39,  44,  64,  63,
       -15, -59,  28,  54,  51,  53,  27,  35,  65,  75,  42,  33,  60,
       -45, -20, -13, -39,  57,  36,  37,  62,  -9,  79,  81,  68,  67,
        58,  46,  40,  43,  49,  47,  55,  69,  56,  61, -26, -14,  98,
        59,  74, -21, -61,  66,  71], dtype=int64)

In [27]:
plays['passResult'].value_counts()

C    5634
R     471
Name: passResult, dtype: int64

In [28]:
plays['playNullifiedByPenalty'].value_counts()

N    12149
Y      337
Name: playNullifiedByPenalty, dtype: int64

In [29]:
plays['defendersInTheBox'].value_counts()

6.0     5500
7.0     3692
8.0     1457
5.0     1238
4.0      319
9.0      166
3.0       51
10.0      35
11.0      19
2.0        3
1.0        1
Name: defendersInTheBox, dtype: int64

There are 3974 unique playId's and many have duplicate values. That means many playId's are repeated for different games. So the new gamePlayId now matches the actual number of rows in the dataframe, as each row is a unique play for each game. 

In [30]:
plays['playId'].nunique()

3974

In [31]:
plays['playId'].value_counts()

56      39
77      15
633     12
78      12
55      11
        ..
4905     1
2484     1
3474     1
3726     1
2509     1
Name: playId, Length: 3974, dtype: int64

In [32]:
plays['gameId'].nunique()

136

In [33]:
plays['gamePlayId'].nunique()

12486

In [34]:
plays['playDescription'].nunique()

12486

In [35]:
tackles

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
1,2022090800,393,46232,1,0,0,0
2,2022090800,486,40166,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0
...,...,...,...,...,...,...,...
17421,2022091113,2494,43533,0,0,0,1
17422,2022092502,3510,42406,0,0,0,1
17423,2022091113,3642,43478,0,0,0,1
17424,2022091901,3578,42431,0,0,0,1


In [36]:
tackles['gamePlayId'] = tackles[['gameId', 'playId']].astype(str).apply(lambda x: ''.join(x), axis=1)
tackles['gamePlayId'] = tackles['gamePlayId'].astype('int64')
tackles['tackleDF'] = 1
tackles.rename(columns={'tackle':'tackler'}, inplace=True)

In [37]:
tackles = pd.merge(tackles, games_weeks, on=['gameId'], how='left')

In [38]:
tackles

,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,gamePlayId,tackleDF,week
0,2022090800,101,42816,1,0,0,0,2022090800101,1,1
1,2022090800,393,46232,1,0,0,0,2022090800393,1,1
2,2022090800,486,40166,1,0,0,0,2022090800486,1,1
3,2022090800,646,47939,1,0,0,0,2022090800646,1,1
4,2022090800,818,40107,1,0,0,0,2022090800818,1,1
...,...,...,...,...,...,...,...,...,...,...
17421,2022091113,2494,43533,0,0,0,1,20220911132494,1,1
17422,2022092502,3510,42406,0,0,0,1,20220925023510,1,3
17423,2022091113,3642,43478,0,0,0,1,20220911133642,1,1
17424,2022091901,3578,42431,0,0,0,1,20220919013578,1,2


In [39]:
tackles['pff_missedTackle'].value_counts()

0    15336
1     2090
Name: pff_missedTackle, dtype: int64

In [40]:
tackles['forcedFumble'].value_counts()

0    17327
1       99
Name: forcedFumble, dtype: int64

In [41]:
tackles['tackler'].value_counts()

1    9919
0    7507
Name: tackler, dtype: int64

In [42]:
tackles['assist'].value_counts()

0    11932
1     5494
Name: assist, dtype: int64

In [43]:
week1

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022090800,56,35472.0,Rodger Saffold,1,2022-09-08 20:24:05.200000,76.0,BUF,left,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,NaN
1,2022090800,56,35472.0,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived
2,2022090800,56,35472.0,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN
3,2022090800,56,35472.0,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN
4,2022090800,56,35472.0,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,2022-09-12 23:05:57.799999,NaN,football,left,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle
1407435,2022091200,3826,NaN,football,50,2022-09-12 23:05:57.900000,NaN,football,left,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN
1407436,2022091200,3826,NaN,football,51,2022-09-12 23:05:58.000000,NaN,football,left,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN
1407437,2022091200,3826,NaN,football,52,2022-09-12 23:05:58.099999,NaN,football,left,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN


In [44]:
week1['gamePlayId'] = week1[['gameId', 'playId']].astype(str).apply(lambda x: ''.join(x), axis=1)
week1['gamePlayId'] = week1['gamePlayId'].astype('int64')

In [45]:
print(week1.columns.tolist())

['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'time', 'jerseyNumber', 'club', 'playDirection', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event', 'gamePlayId']


In [46]:
week1['event'].value_counts()

first_contact                28773
tackle                       26928
ball_snap                    16415
pass_outcome_caught          15870
handoff                      15364
pass_arrived                 13915
out_of_bounds                 5037
run                           2737
man_in_motion                 1288
play_action                   1035
touchdown                     1012
fumble                         621
shift                          368
qb_slide                       350
pass_forward                   248
snap_direct                     46
line_set                        46
lateral                         45
autoevent_ballsnap              30
run_pass_option                 23
qb_sack                         23
pass_shovel                     23
fumble_defense_recovered        23
fumble_offense_recovered        23
autoevent_passinterrupted       16
autoevent_passforward            9
Name: event, dtype: int64

In [47]:
week1.drop(columns=['time', 'jerseyNumber', 'playDirection'], inplace=True)

 To removed rows where the player name is football.

The playId values are not unique to each game, so the gameId and playId are combined to create a new gamePlayId value that will be unique to each play for each game. 

In [48]:
week1.dtypes

gameId           int64
playId           int64
nflId          float64
displayName     object
frameId          int64
club            object
x              float64
y              float64
s              float64
a              float64
dis            float64
o              float64
dir            float64
event           object
gamePlayId       int64
dtype: object

In [49]:
week1[0:30]

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId
0,2022090800,56,35472.0,Rodger Saffold,1,BUF,88.37,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056
1,2022090800,56,35472.0,Rodger Saffold,2,BUF,88.47,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056
2,2022090800,56,35472.0,Rodger Saffold,3,BUF,88.56,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056
3,2022090800,56,35472.0,Rodger Saffold,4,BUF,88.64,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056
4,2022090800,56,35472.0,Rodger Saffold,5,BUF,88.72,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056
5,2022090800,56,35472.0,Rodger Saffold,6,BUF,88.80,26.70,1.15,1.42,0.12,234.48,139.41,pass_outcome_caught,202209080056
6,2022090800,56,35472.0,Rodger Saffold,7,BUF,88.87,26.64,0.93,1.69,0.09,235.77,134.32,NaN,202209080056
7,2022090800,56,35472.0,Rodger Saffold,8,BUF,88.91,26.59,0.68,1.74,0.07,240.00,131.01,NaN,202209080056
8,2022090800,56,35472.0,Rodger Saffold,9,BUF,88.94,26.57,0.42,1.74,0.04,243.56,122.29,NaN,202209080056
9,2022090800,56,35472.0,Rodger Saffold,10,BUF,88.95,26.58,0.14,1.83,0.01,246.07,85.87,NaN,202209080056


In [50]:
print(week1.columns.tolist())

['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'club', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event', 'gamePlayId']


In [51]:
week1['playId'].nunique()

1247

In [52]:
week1['gamePlayId'].nunique()

1473

In [53]:
week1['gameId'].nunique()

16

In [54]:
week1['frameId'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114], dtype=int64)

In [55]:
week1['event'].value_counts()

first_contact                28773
tackle                       26928
ball_snap                    16415
pass_outcome_caught          15870
handoff                      15364
pass_arrived                 13915
out_of_bounds                 5037
run                           2737
man_in_motion                 1288
play_action                   1035
touchdown                     1012
fumble                         621
shift                          368
qb_slide                       350
pass_forward                   248
snap_direct                     46
line_set                        46
lateral                         45
autoevent_ballsnap              30
run_pass_option                 23
qb_sack                         23
pass_shovel                     23
fumble_defense_recovered        23
fumble_offense_recovered        23
autoevent_passinterrupted       16
autoevent_passforward            9
Name: event, dtype: int64

In [56]:
week1 = pd.merge(week1, tackles, on=['gameId','playId','nflId','gamePlayId'], how='left')

In [57]:
week1 = week1.drop('week', axis=1)

In [58]:
week1['assist'].value_counts()

0.0    64312
1.0    28689
Name: assist, dtype: int64

To look at rows where there were tackles to identify plays where there were no tackles, so they can be eliminated. 

In [59]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF
0,2022090800,56,35472.0,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,NaN
1,2022090800,56,35472.0,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,NaN
2,2022090800,56,35472.0,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,NaN
3,2022090800,56,35472.0,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,NaN
4,2022090800,56,35472.0,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,football,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN
1407435,2022091200,3826,NaN,football,50,football,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN
1407436,2022091200,3826,NaN,football,51,football,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN
1407437,2022091200,3826,NaN,football,52,football,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN


In [60]:
week1 = pd.merge(week1, players, on=['nflId','displayName'], how='left')

In [61]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,forcedFumble,pff_missedTackle,tackleDF,height,weight,birthDate,collegeName,position,totalInches,mass
0,2022090800,56,35472.0,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,NaN,NaN,NaN,6-5,325.0,1988-06-06,Indiana,G,77.0,2085.42
1,2022090800,56,35472.0,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,NaN,NaN,NaN,6-5,325.0,1988-06-06,Indiana,G,77.0,2085.42
2,2022090800,56,35472.0,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,NaN,NaN,NaN,6-5,325.0,1988-06-06,Indiana,G,77.0,2085.42
3,2022090800,56,35472.0,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,NaN,NaN,NaN,6-5,325.0,1988-06-06,Indiana,G,77.0,2085.42
4,2022090800,56,35472.0,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,NaN,NaN,NaN,6-5,325.0,1988-06-06,Indiana,G,77.0,2085.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,football,56.220001,9.89,2.56,1.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1407435,2022091200,3826,NaN,football,50,football,56.060001,10.08,2.50,1.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1407436,2022091200,3826,NaN,football,51,football,55.889999,10.27,2.38,1.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1407437,2022091200,3826,NaN,football,52,football,55.730000,10.44,2.07,2.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
week1.drop(['height', 'weight', 'birthDate', 'collegeName'], axis=1, inplace=True)

In [63]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass
0,2022090800,56,35472.0,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42
1,2022090800,56,35472.0,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,pass_arrived,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42
2,2022090800,56,35472.0,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42
3,2022090800,56,35472.0,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42
4,2022090800,56,35472.0,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,football,56.220001,9.89,2.56,1.25,...,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1407435,2022091200,3826,NaN,football,50,football,56.060001,10.08,2.50,1.14,...,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1407436,2022091200,3826,NaN,football,51,football,55.889999,10.27,2.38,1.70,...,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1407437,2022091200,3826,NaN,football,52,football,55.730000,10.44,2.07,2.83,...,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
week1['position'].value_counts()

CB     154624
WR     151571
T      126873
G      115431
OLB    100436
DT      88019
DE      82436
ILB     81620
TE      81513
FS      77900
C       65067
RB      62904
QB      61193
SS      57195
NT      20887
FB       7194
MLB      7027
DB       1937
Name: position, dtype: int64

In [65]:
week1_events = week1[week1['event'].notna()]

In [66]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass
1,2022090800,56,35472.0,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,pass_arrived,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42
5,2022090800,56,35472.0,Rodger Saffold,6,BUF,88.800000,26.70,1.15,1.42,...,pass_outcome_caught,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42
17,2022090800,56,35472.0,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,...,tackle,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42
23,2022090800,56,38577.0,Bobby Wagner,2,LA,78.200000,27.69,1.78,4.43,...,pass_arrived,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00
27,2022090800,56,38577.0,Bobby Wagner,6,LA,78.110000,28.74,3.35,2.62,...,pass_outcome_caught,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618.0,Tariq Woolen,49,SEA,50.750000,6.92,1.88,2.49,...,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,CB,76.0,1298.33
1407389,2022091200,3826,NaN,football,4,football,70.860001,13.25,5.55,0.46,...,pass_arrived,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1407391,2022091200,3826,NaN,football,6,football,70.529999,12.14,5.55,1.14,...,pass_outcome_caught,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1407409,2022091200,3826,NaN,football,24,football,64.459999,5.05,5.21,4.07,...,first_contact,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Comparing Tackling Data to Events with Tackles

In [67]:
week1_events_merge = week1_events[['gamePlayId', 'event']]

In [68]:
week1_events_merge

,gamePlayId,event
1,202209080056,pass_arrived
5,202209080056,pass_outcome_caught
17,202209080056,tackle
23,202209080056,pass_arrived
27,202209080056,pass_outcome_caught
...,...,...
1407381,20220912003826,tackle
1407389,20220912003826,pass_arrived
1407391,20220912003826,pass_outcome_caught
1407409,20220912003826,first_contact


In [69]:
week1_events_merge['event'].value_counts()

first_contact                28773
tackle                       26928
ball_snap                    16415
pass_outcome_caught          15870
handoff                      15364
pass_arrived                 13915
out_of_bounds                 5037
run                           2737
man_in_motion                 1288
play_action                   1035
touchdown                     1012
fumble                         621
shift                          368
qb_slide                       350
pass_forward                   248
snap_direct                     46
line_set                        46
lateral                         45
autoevent_ballsnap              30
run_pass_option                 23
qb_sack                         23
pass_shovel                     23
fumble_defense_recovered        23
fumble_offense_recovered        23
autoevent_passinterrupted       16
autoevent_passforward            9
Name: event, dtype: int64

In [70]:
drop_events = ['run', 'man_in_motion', 'play_action', 'shift', 'qb_slide', 'pass_forward', 'touchdown', 'lateral', 
               'line_set', 'snap_direct', 'autoevent_ballsnap', 'fumble_offense_recovered', 'run_pass_option', 
               'pass_shovel', 'fumble_defense_recovered', 'qb_sack']

In [71]:
week1_events_merge = week1_events_merge[~week1_events_merge.event.isin(drop_events)]

In [72]:
week1_events_merge

,gamePlayId,event
1,202209080056,pass_arrived
5,202209080056,pass_outcome_caught
17,202209080056,tackle
23,202209080056,pass_arrived
27,202209080056,pass_outcome_caught
...,...,...
1407381,20220912003826,tackle
1407389,20220912003826,pass_arrived
1407391,20220912003826,pass_outcome_caught
1407409,20220912003826,first_contact


In [73]:
event_dummies = pd.get_dummies(week1_events_merge['event'])

In [74]:
event_dummies

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle
1,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,0,0,0,1,0
17,0,0,0,0,0,0,0,0,0,1
23,0,0,0,0,0,0,0,1,0,0
27,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
1407381,0,0,0,0,0,0,0,0,0,1
1407389,0,0,0,0,0,0,0,1,0,0
1407391,0,0,0,0,0,0,0,0,1,0
1407409,0,0,0,1,0,0,0,0,0,0


In [75]:
week1_events_merge = pd.concat([week1_events_merge, event_dummies], axis=1) 
week1_events_merge.drop(['event'], inplace=True, axis=1)

In [76]:
week1_events_merge

,gamePlayId,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle
1,202209080056,0,0,0,0,0,0,0,1,0,0
5,202209080056,0,0,0,0,0,0,0,0,1,0
17,202209080056,0,0,0,0,0,0,0,0,0,1
23,202209080056,0,0,0,0,0,0,0,1,0,0
27,202209080056,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1407381,20220912003826,0,0,0,0,0,0,0,0,0,1
1407389,20220912003826,0,0,0,0,0,0,0,1,0,0
1407391,20220912003826,0,0,0,0,0,0,0,0,1,0
1407409,20220912003826,0,0,0,1,0,0,0,0,0,0


In [77]:
week1_events_merge.apply(pd.Series.value_counts)

,gamePlayId,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle
0,NaN,122939.0,122932.0,106533.0,94175.0,122327.0,107584.0,117911.0,109033.0,107078.0,96020.0
1,NaN,9.0,16.0,16415.0,28773.0,621.0,15364.0,5037.0,13915.0,15870.0,26928.0
202209080056,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202209080080,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202209110057,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20220912003628,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20220912003685,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20220912003723,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20220912003747,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
events_keep = ['ball_snap', 'first_contact', 'fumble', 'handoff', 'out_of_bounds', 'pass_arrived', 'pass_outcome_caught', 
               'tackle']

In [79]:
week1_events_merge['ball_snap'] = week1_events_merge.groupby(['gamePlayId'])['ball_snap'].transform(max)

In [80]:
for i in events_keep:
    week1_events_merge[i] = week1_events_merge.groupby(['gamePlayId'])[i].transform(max)

In [81]:
week1_events_merge

,gamePlayId,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle
1,202209080056,0,0,0,0,0,0,0,1,1,1
5,202209080056,0,0,0,0,0,0,0,1,1,1
17,202209080056,0,0,0,0,0,0,0,1,1,1
23,202209080056,0,0,0,0,0,0,0,1,1,1
27,202209080056,0,0,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1407381,20220912003826,0,0,0,1,0,0,0,1,1,1
1407389,20220912003826,0,0,0,1,0,0,0,1,1,1
1407391,20220912003826,0,0,0,1,0,0,0,1,1,1
1407409,20220912003826,0,0,0,1,0,0,0,1,1,1


In [82]:
week1_events_merge.apply(pd.Series.value_counts)

,gamePlayId,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle
0,NaN,122939.0,122932.0,59878.0,12379.0,120712.0,62870.0,105238.0,70507.0,65317.0,22107.0
1,NaN,9.0,16.0,63070.0,110569.0,2236.0,60078.0,17710.0,52441.0,57631.0,100841.0
202209080056,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202209080080,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202209110057,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20220912003628,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20220912003685,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20220912003723,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20220912003747,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
week1_events_merge['gamePlayId'].nunique()

1465

In [84]:
test_groupby = week1_events_merge.groupby('gamePlayId').first()

In [85]:
test_groupby

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle
gamePlayId,,,,,,,,,,
202209080056,0,0,0,0,0,0,0,1,1,1
202209080080,0,0,0,1,0,0,0,0,0,1
202209110057,0,0,0,1,0,0,0,0,0,1
202209110078,0,0,1,1,0,1,0,0,0,1
202209110185,0,0,0,1,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
20220912003628,0,0,0,1,0,0,0,1,1,1
20220912003685,0,0,1,1,0,1,0,0,0,1
20220912003723,0,0,0,1,0,0,0,1,1,1


In [86]:
test_groupby.apply(pd.Series.value_counts)

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle
0,1465.0,1465.0,751,214,1438,802,1246,861,775,294
1,NaN,NaN,714,1251,27,663,219,604,690,1171


In [87]:
tackles_week1 = tackles[tackles.week == 1]

In [88]:
tackles_week1

,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,gamePlayId,tackleDF,week
0,2022090800,101,42816,1,0,0,0,2022090800101,1,1
1,2022090800,393,46232,1,0,0,0,2022090800393,1,1
2,2022090800,486,40166,1,0,0,0,2022090800486,1,1
3,2022090800,646,47939,1,0,0,0,2022090800646,1,1
4,2022090800,818,40107,1,0,0,0,2022090800818,1,1
...,...,...,...,...,...,...,...,...,...,...
17406,2022091109,3986,44880,0,0,0,1,20220911093986,1,1
17407,2022091109,4075,54466,0,0,0,1,20220911094075,1,1
17408,2022091109,4096,43787,0,0,0,1,20220911094096,1,1
17421,2022091113,2494,43533,0,0,0,1,20220911132494,1,1


In [89]:
tackles_week1['gamePlayId'].value_counts()

20220911062840    4
2022091101641     4
20220911054905    4
20220911123416    4
20220911083251    4
                 ..
20220911111172    1
2022091111899     1
2022091111542     1
2022091111451     1
20220911011616    1
Name: gamePlayId, Length: 1421, dtype: int64

In [90]:
tackles_list = ['tackler', 'assist', 'forcedFumble', 'pff_missedTackle']

In [91]:
for i in tackles_list:
    tackles_week1[i] = tackles_week1.groupby(['gamePlayId'])[i].transform(max)

In [92]:
tackles_week1

,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,gamePlayId,tackleDF,week
0,2022090800,101,42816,1,0,0,0,2022090800101,1,1
1,2022090800,393,46232,1,0,0,0,2022090800393,1,1
2,2022090800,486,40166,1,0,0,0,2022090800486,1,1
3,2022090800,646,47939,1,0,0,0,2022090800646,1,1
4,2022090800,818,40107,1,1,0,0,2022090800818,1,1
...,...,...,...,...,...,...,...,...,...,...
17406,2022091109,3986,44880,1,0,0,1,20220911093986,1,1
17407,2022091109,4075,54466,1,0,0,1,20220911094075,1,1
17408,2022091109,4096,43787,1,0,0,1,20220911094096,1,1
17421,2022091113,2494,43533,1,0,0,1,20220911132494,1,1


In [93]:
tackles_week1 = tackles_week1.groupby('gamePlayId').first()

In [94]:
tackles_week1

,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
gamePlayId,,,,,,,,,
202209080056,2022090800,56,43294,1,0,0,0,1,1
202209080080,2022090800,80,53532,1,0,0,0,1,1
202209110057,2022091100,57,53489,1,0,0,0,1,1
202209110078,2022091100,78,40017,1,0,0,1,1,1
202209110185,2022091101,85,44974,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...
20220912003628,2022091200,3628,47871,1,0,0,1,1,1
20220912003685,2022091200,3685,35562,1,1,0,0,1,1
20220912003723,2022091200,3723,52435,1,1,0,1,1,1


In [95]:
test_groupby = pd.merge(test_groupby, tackles_week1, on='gamePlayId', how='left')

In [96]:
test_groupby

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
gamePlayId,,,,,,,,,,,,,,,,,,,
202209080056,0,0,0,0,0,0,0,1,1,1,2.022091e+09,56.0,43294.0,1.0,0.0,0.0,0.0,1.0,1.0
202209080080,0,0,0,1,0,0,0,0,0,1,2.022091e+09,80.0,53532.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110057,0,0,0,1,0,0,0,0,0,1,2.022091e+09,57.0,53489.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110078,0,0,1,1,0,1,0,0,0,1,2.022091e+09,78.0,40017.0,1.0,0.0,0.0,1.0,1.0,1.0
202209110185,0,0,0,1,0,0,0,1,1,1,2.022091e+09,85.0,44974.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220912003628,0,0,0,1,0,0,0,1,1,1,2.022091e+09,3628.0,47871.0,1.0,0.0,0.0,1.0,1.0,1.0
20220912003685,0,0,1,1,0,1,0,0,0,1,2.022091e+09,3685.0,35562.0,1.0,1.0,0.0,0.0,1.0,1.0
20220912003723,0,0,0,1,0,0,0,1,1,1,2.022091e+09,3723.0,52435.0,1.0,1.0,0.0,1.0,1.0,1.0


In [97]:
test_groupby.apply(pd.Series.value_counts)

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
0.000000e+00,1465.0,1465.0,751.0,214.0,1438.0,802.0,1246.0,861.0,775.0,294.0,NaN,NaN,NaN,207.0,1028.0,1390.0,1199.0,NaN,NaN
1.000000e+00,NaN,NaN,714.0,1251.0,27.0,663.0,219.0,604.0,690.0,1171.0,NaN,NaN,NaN,1208.0,387.0,25.0,216.0,1415.0,1415.0
5.500000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.600000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.700000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
test_groupby_notackle = test_groupby[test_groupby.tackle == 0]

In [99]:
test_groupby_notackle

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
gamePlayId,,,,,,,,,,,,,,,,,,,
202209110382,0,0,0,0,0,0,1,1,1,0,2.022091e+09,82.0,43641.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110883,0,0,1,0,0,0,0,0,0,0,2.022091e+09,83.0,47899.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110991,0,0,1,1,0,1,1,0,0,0,2.022091e+09,91.0,47956.0,1.0,0.0,0.0,1.0,1.0,1.0
202209111076,0,0,0,0,0,0,1,0,0,0,2.022091e+09,76.0,53565.0,1.0,0.0,0.0,0.0,1.0,1.0
2022090800236,0,0,0,0,0,0,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220912002923,0,0,1,1,0,0,1,0,0,0,2.022091e+09,2923.0,42393.0,1.0,1.0,0.0,0.0,1.0,1.0
20220912003048,0,0,0,1,0,0,1,1,1,0,2.022091e+09,3048.0,47871.0,1.0,0.0,0.0,0.0,1.0,1.0
20220912003216,0,0,0,0,0,0,1,1,1,0,2.022091e+09,3216.0,44873.0,1.0,0.0,0.0,0.0,1.0,1.0


There were 294 plays (20%) in week 1 where the tackle event was 0. Of those, 219 plays (75%) had an out of bounds event. 

So it appears most of these plays without a tackle event were plays ending with the runner running out of bounds. Out of the total of 1465 plays in week 1, only 75 (5%) did not have a tackle event and did not go out of bounds. 

Therefore, since we are primarily studying tackling, we only need to include those plays that had a tackle event occuring in our analysis. 

Another interesting note is that in the overall 1465 plays only 714 (49%) had a snap event. For those with snaps, what was the average frame the snap occured? Possible that frame could be used on non-snap event plays in place of a snap event. 

In [100]:
test_groupby_notackle.apply(pd.Series.value_counts)

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
0.000000e+00,294.0,294.0,208.0,96.0,268.0,224.0,75.0,122.0,107.0,294.0,NaN,NaN,NaN,11.0,232.0,221.0,188.0,NaN,NaN
1.000000e+00,NaN,NaN,86.0,198.0,26.0,70.0,219.0,172.0,187.0,NaN,NaN,NaN,NaN,234.0,13.0,24.0,57.0,245.0,245.0
7.600000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.200000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.300000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
test_groupby_tackler = test_groupby[test_groupby.tackler == 1]

In [102]:
test_groupby_tackler

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
gamePlayId,,,,,,,,,,,,,,,,,,,
202209080056,0,0,0,0,0,0,0,1,1,1,2.022091e+09,56.0,43294.0,1.0,0.0,0.0,0.0,1.0,1.0
202209080080,0,0,0,1,0,0,0,0,0,1,2.022091e+09,80.0,53532.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110057,0,0,0,1,0,0,0,0,0,1,2.022091e+09,57.0,53489.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110078,0,0,1,1,0,1,0,0,0,1,2.022091e+09,78.0,40017.0,1.0,0.0,0.0,1.0,1.0,1.0
202209110185,0,0,0,1,0,0,0,1,1,1,2.022091e+09,85.0,44974.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220912003628,0,0,0,1,0,0,0,1,1,1,2.022091e+09,3628.0,47871.0,1.0,0.0,0.0,1.0,1.0,1.0
20220912003685,0,0,1,1,0,1,0,0,0,1,2.022091e+09,3685.0,35562.0,1.0,1.0,0.0,0.0,1.0,1.0
20220912003723,0,0,0,1,0,0,0,1,1,1,2.022091e+09,3723.0,52435.0,1.0,1.0,0.0,1.0,1.0,1.0


In [103]:
test_groupby_tackler.apply(pd.Series.value_counts)

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
0.000000e+00,1208.0,1208.0,650.0,172.0,1183.0,693.0,1005.0,693.0,616.0,234.0,NaN,NaN,NaN,NaN,1021.0,1184.0,1029.0,NaN,NaN
1.000000e+00,NaN,NaN,558.0,1036.0,25.0,515.0,203.0,515.0,592.0,974.0,NaN,NaN,NaN,1208.0,187.0,24.0,179.0,1208.0,1208.0
5.600000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.700000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.800000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
test_groupby_tackleEvent = test_groupby[test_groupby.tackle == 1]

In [105]:
test_groupby_tackleEvent

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
gamePlayId,,,,,,,,,,,,,,,,,,,
202209080056,0,0,0,0,0,0,0,1,1,1,2.022091e+09,56.0,43294.0,1.0,0.0,0.0,0.0,1.0,1.0
202209080080,0,0,0,1,0,0,0,0,0,1,2.022091e+09,80.0,53532.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110057,0,0,0,1,0,0,0,0,0,1,2.022091e+09,57.0,53489.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110078,0,0,1,1,0,1,0,0,0,1,2.022091e+09,78.0,40017.0,1.0,0.0,0.0,1.0,1.0,1.0
202209110185,0,0,0,1,0,0,0,1,1,1,2.022091e+09,85.0,44974.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220912003628,0,0,0,1,0,0,0,1,1,1,2.022091e+09,3628.0,47871.0,1.0,0.0,0.0,1.0,1.0,1.0
20220912003685,0,0,1,1,0,1,0,0,0,1,2.022091e+09,3685.0,35562.0,1.0,1.0,0.0,0.0,1.0,1.0
20220912003723,0,0,0,1,0,0,0,1,1,1,2.022091e+09,3723.0,52435.0,1.0,1.0,0.0,1.0,1.0,1.0


In [106]:
test_groupby_tackleEvent.apply(pd.Series.value_counts)

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
0.000000e+00,1171.0,1171.0,543.0,118.0,1170.0,578.0,1171.0,739.0,668.0,NaN,NaN,NaN,NaN,196.0,796.0,1169.0,1011.0,NaN,NaN
1.000000e+00,NaN,NaN,628.0,1053.0,1.0,593.0,NaN,432.0,503.0,1171.0,NaN,NaN,NaN,974.0,374.0,1.0,159.0,1170.0,1170.0
5.500000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.600000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.700000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


When there is a tackler identified to a play, 234 out of 1,208 plays (19%) don't have a tackle event. 203 of these plays were out of bounds. Does that mostly explain why? Possibly consider plays with no tackle event, but an out of bounds event at the out of bound event being the last event in the yards calculation. 

One the other hand, when there is a tackle event 196 out of 1171 plays (17%) do not have a tackler identified. Should the closest defensive player(s) to the ball then get a tackler designation? None of these 1171 plays have an out of bounds event. 

# What is Average Frame for Snaps on Plays with Snap Events?

In [309]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle,snapToTransfer,transferToContact,transferToTackle,contactToTackle,snapToTackle,snapToContact,xBallSnap,xBallTransfer,xBallContact,xBallTackle,ballSnapToTransfer,ballSnapToContact,ballTransferToContact,ballContactToTackle,ballSnapToTackle,ballTransferToTackle,ballTackleToEnd
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.800000,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
22,2022090800,56,38577,Bobby Wagner,1,LA,78.250000,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
27,2022090800,56,38577,Bobby Wagner,6,LA,78.110000,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.750000,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45,2441.0,3233.0,223229.0,295658.0,0,0,0,0,1,0,53.280000,15.53,48.190000,13.00,51.220000,16.13,50.750000,6.92,2.145600,4.356352,9.221985,6.596969,8.974018,5.684101,70.71,70.57,64.41,56.06,0.14,6.30,6.16,8.35,14.65,14.51,0.34
1407386,2022091200,3826,0,football,1,football,71.400002,17.91,20.33,3.93,2.05,NaN,NaN,snap,20220912

In [108]:
week1_events.nunique()

gameId                 16
playId               1247
nflId                1171
displayName          1166
frameId                88
club                   33
x                   14962
y                    8491
s                    2151
a                    1830
dis                   390
o                   34627
dir                 34758
event                  26
gamePlayId           1473
tackler                 2
assist                  2
forcedFumble            2
pff_missedTackle        2
tackleDF                1
position               18
totalInches            16
mass                  573
dtype: int64

In [109]:
week1_events_snap = week1_events[['gamePlayId', 'frameId', 'event']]

In [110]:
week1_events_snap

,gamePlayId,frameId,event
1,202209080056,2,pass_arrived
5,202209080056,6,pass_outcome_caught
17,202209080056,18,tackle
23,202209080056,2,pass_arrived
27,202209080056,6,pass_outcome_caught
...,...,...,...
1407381,20220912003826,49,tackle
1407389,20220912003826,4,pass_arrived
1407391,20220912003826,6,pass_outcome_caught
1407409,20220912003826,24,first_contact


In [111]:
week1_events_snap = week1_events_snap.loc[week1_events_snap['event'] == 'ball_snap']

In [112]:
week1_events_snap

,gamePlayId,frameId,event
1201,2022090800101,6,ball_snap
1250,2022090800101,6,ball_snap
1299,2022090800101,6,ball_snap
1348,2022090800101,6,ball_snap
1397,2022090800101,6,ball_snap
...,...,...,...
1403934,20220912003685,6,ball_snap
1403992,20220912003685,6,ball_snap
1404050,20220912003685,6,ball_snap
1404108,20220912003685,6,ball_snap


In [113]:
week1_events_snap['frameId'].mean()

6.009808102345416

In [114]:
week1_events_snap['frameId'].std()

0.2618486098874191

In [115]:
week1_events_snap['frameId'].median()

6.0

In [116]:
print (len(week1[week1['frameId'] == 6]))

33879


In [117]:
week1['frameId'].value_counts().nlargest(15)

1     33879
2     33879
3     33879
4     33879
5     33879
6     33879
7     33879
8     33879
9     33879
10    33879
11    33879
12    33488
13    33074
14    32545
15    32062
Name: frameId, dtype: int64

It appears frame 6 only occurs 16,720 times in week 1, whereas the other first 11 frames each occur 32,406 times. So it would then be best to set all snaps at either frame 5 or 7. 

# Are Runs and Passes Completely Separate Events?

In [118]:
test_groupby_run = test_groupby[test_groupby.handoff == 1]

In [119]:
test_groupby_run

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
gamePlayId,,,,,,,,,,,,,,,,,,,
202209110078,0,0,1,1,0,1,0,0,0,1,2.022091e+09,78.0,40017.0,1.0,0.0,0.0,1.0,1.0,1.0
202209110286,0,0,1,1,0,1,0,0,0,1,2.022091e+09,86.0,44938.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110486,0,0,1,1,0,1,0,0,0,1,2.022091e+09,86.0,39984.0,1.0,0.0,0.0,1.0,1.0,1.0
202209110556,0,0,1,1,0,1,0,0,0,1,2.022091e+09,56.0,52498.0,0.0,1.0,0.0,0.0,1.0,1.0
202209110683,0,0,1,1,0,1,0,0,0,1,2.022091e+09,83.0,48476.0,1.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220912002944,0,0,1,1,0,1,0,0,0,1,2.022091e+09,2944.0,45011.0,1.0,0.0,0.0,0.0,1.0,1.0
20220912003077,0,0,1,1,0,1,0,0,0,1,2.022091e+09,3077.0,54618.0,1.0,0.0,0.0,0.0,1.0,1.0
20220912003173,0,0,1,1,0,1,0,0,0,1,2.022091e+09,3173.0,44855.0,1.0,0.0,0.0,1.0,1.0,1.0


In [120]:
test_groupby_run.apply(pd.Series.value_counts)

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
0.000000e+00,663.0,663.0,1.0,33.0,652.0,NaN,618.0,662.0,661.0,70.0,NaN,NaN,NaN,138.0,405.0,642.0,538.0,NaN,NaN
1.000000e+00,NaN,NaN,662.0,630.0,11.0,663.0,45.0,1.0,2.0,593.0,NaN,NaN,NaN,515.0,248.0,11.0,115.0,653.0,653.0
5.500000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.600000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6.200000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
test_groupby_pass = test_groupby[test_groupby.pass_outcome_caught == 1]

In [122]:
test_groupby_pass

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
gamePlayId,,,,,,,,,,,,,,,,,,,
202209080056,0,0,0,0,0,0,0,1,1,1,2.022091e+09,56.0,43294.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110185,0,0,0,1,0,0,0,1,1,1,2.022091e+09,85.0,44974.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110358,0,0,0,1,0,0,0,1,1,1,2.022091e+09,58.0,52473.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110382,0,0,0,0,0,0,1,1,1,0,2.022091e+09,82.0,43641.0,1.0,0.0,0.0,0.0,1.0,1.0
202209110458,0,0,0,1,0,0,0,1,1,1,2.022091e+09,58.0,46188.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220912003596,0,0,0,1,0,0,0,1,1,1,2.022091e+09,3596.0,42543.0,1.0,0.0,0.0,0.0,1.0,1.0
20220912003628,0,0,0,1,0,0,0,1,1,1,2.022091e+09,3628.0,47871.0,1.0,0.0,0.0,1.0,1.0,1.0
20220912003723,0,0,0,1,0,0,0,1,1,1,2.022091e+09,3723.0,52435.0,1.0,1.0,0.0,1.0,1.0,1.0


In [123]:
test_groupby_pass.apply(pd.Series.value_counts)

,autoevent_passforward,autoevent_passinterrupted,ball_snap,first_contact,fumble,handoff,out_of_bounds,pass_arrived,pass_outcome_caught,tackle,gameId,playId,nflId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,week
0.000000e+00,690.0,690.0,688.0,152.0,677.0,688.0,538.0,86.0,NaN,187.0,NaN,NaN,NaN,61.0,535.0,640.0,575.0,NaN,NaN
1.000000e+00,NaN,NaN,2.0,538.0,13.0,2.0,152.0,604.0,690.0,503.0,NaN,NaN,NaN,592.0,118.0,13.0,78.0,653.0,653.0
5.600000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.800000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.900000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.022091e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are 663 plays where there is a run event. Of those plays, only 1 has an pass arrive event and only 2 have a pass caught event. 

There are 690 plays where there is a pass caught event. Of those plays, only 2 plays have a handoff event. 

In summary, it appears the run and pass events are completely separate, as should be expected.  

Another interesting datapoint is that of the 663 run plays, 662 of those plays have a snap event. On the other hand, of the 690 pass plays, only 2 have a snap event. It appears that snaps are only recorded on running plays, not passing plays. This reinforces the decision to simply use frame number 6 as the snap frame on all plays. 

In [124]:
week1['event'].value_counts()

first_contact                28773
tackle                       26928
ball_snap                    16415
pass_outcome_caught          15870
handoff                      15364
pass_arrived                 13915
out_of_bounds                 5037
run                           2737
man_in_motion                 1288
play_action                   1035
touchdown                     1012
fumble                         621
shift                          368
qb_slide                       350
pass_forward                   248
snap_direct                     46
line_set                        46
lateral                         45
autoevent_ballsnap              30
run_pass_option                 23
qb_sack                         23
pass_shovel                     23
fumble_defense_recovered        23
fumble_offense_recovered        23
autoevent_passinterrupted       16
autoevent_passforward            9
Name: event, dtype: int64

Before changing both handoffs and passes caught to a transfer event, a new column 'playType' will identify pass vs run plays based on these two event types. 

In [125]:
week1['playType'] = np.where(week1['event']=='handoff', 'run', 0)

In [126]:
week1['playType'] = np.where(week1['event']=='pass_outcome_caught', 'pass', week1['playType'])

In [127]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType
0,2022090800,56,35472.0,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,0
1,2022090800,56,35472.0,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,0
2,2022090800,56,35472.0,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,0
3,2022090800,56,35472.0,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,0
4,2022090800,56,35472.0,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,football,56.220001,9.89,2.56,1.25,...,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1407435,2022091200,3826,NaN,football,50,football,56.060001,10.08,2.50,1.14,...,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1407436,2022091200,3826,NaN,football,51,football,55.889999,10.27,2.38,1.70,...,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1407437,2022091200,3826,NaN,football,52,football,55.730000,10.44,2.07,2.83,...,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [128]:
week1['playType'].value_counts()

0       1376205
pass      15870
run       15364
Name: playType, dtype: int64

# To Expand Play Types To All Rows in Each gamePlayId Group

In [129]:
week1_playtype = week1[['gamePlayId', 'playType']]

In [130]:
week1_playtype

,gamePlayId,playType
0,202209080056,0
1,202209080056,0
2,202209080056,0
3,202209080056,0
4,202209080056,0
...,...,...
1407434,20220912003826,0
1407435,20220912003826,0
1407436,20220912003826,0
1407437,20220912003826,0


In [131]:
week1_playtype['playType'].value_counts()

0       1376205
pass      15870
run       15364
Name: playType, dtype: int64

In [132]:
grouped_playtype = week1_playtype.groupby(['gamePlayId']).max().reset_index()

In [133]:
grouped_playtype

,gamePlayId,playType
0,202209080056,pass
1,202209080080,0
2,202209110057,0
3,202209110078,run
4,202209110185,pass
...,...,...
1468,20220912003628,pass
1469,20220912003685,run
1470,20220912003723,pass
1471,20220912003747,pass


In [134]:
grouped_playtype['playType'].value_counts()

pass    688
run     663
0       122
Name: playType, dtype: int64

In [135]:
grouped_playtype['playType'] = np.where(grouped_playtype['playType']=='0', 'direct', grouped_playtype['playType'])

In [136]:
grouped_playtype['playType'].value_counts()

pass      688
run       663
direct    122
Name: playType, dtype: int64

In [137]:
grouped_playtype

,gamePlayId,playType
0,202209080056,pass
1,202209080080,direct
2,202209110057,direct
3,202209110078,run
4,202209110185,pass
...,...,...
1468,20220912003628,pass
1469,20220912003685,run
1470,20220912003723,pass
1471,20220912003747,pass


In [138]:
week1 = pd.merge(week1, grouped_playtype, on=['gamePlayId'], how='left')

In [139]:
week1 = week1.drop('playType_x', axis=1)

In [140]:
week1.rename(columns={'playType_y':'playType'}, inplace=True)

In [141]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType
0,2022090800,56,35472.0,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
1,2022090800,56,35472.0,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
2,2022090800,56,35472.0,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
3,2022090800,56,35472.0,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
4,2022090800,56,35472.0,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,football,56.220001,9.89,2.56,1.25,...,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass
1407435,2022091200,3826,NaN,football,50,football,56.060001,10.08,2.50,1.14,...,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass
1407436,2022091200,3826,NaN,football,51,football,55.889999,10.27,2.38,1.70,...,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass
1407437,2022091200,3826,NaN,football,52,football,55.730000,10.44,2.07,2.83,...,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass


In [142]:
week1['playType'].value_counts()

run       837821
pass      444084
direct    125534
Name: playType, dtype: int64

To change the handoff and pass caught events to a transfer event. 

In [143]:
week1_transfer = week1.copy()

In [144]:
week1_transfer['event'] = np.where(week1_transfer['event']=='handoff', 'transfer', week1_transfer['event'])

In [145]:
week1_transfer['event'] = np.where(week1_transfer['event']=='pass_outcome_caught', 'transfer', week1_transfer['event'])

In [146]:
week1_transfer

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType
0,2022090800,56,35472.0,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
1,2022090800,56,35472.0,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
2,2022090800,56,35472.0,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
3,2022090800,56,35472.0,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
4,2022090800,56,35472.0,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,football,56.220001,9.89,2.56,1.25,...,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass
1407435,2022091200,3826,NaN,football,50,football,56.060001,10.08,2.50,1.14,...,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass
1407436,2022091200,3826,NaN,football,51,football,55.889999,10.27,2.38,1.70,...,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass
1407437,2022091200,3826,NaN,football,52,football,55.730000,10.44,2.07,2.83,...,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass


In [147]:
week1_transfer['event'].value_counts()

transfer                     31234
first_contact                28773
tackle                       26928
ball_snap                    16415
pass_arrived                 13915
out_of_bounds                 5037
run                           2737
man_in_motion                 1288
play_action                   1035
touchdown                     1012
fumble                         621
shift                          368
qb_slide                       350
pass_forward                   248
snap_direct                     46
line_set                        46
lateral                         45
autoevent_ballsnap              30
run_pass_option                 23
qb_sack                         23
pass_shovel                     23
fumble_defense_recovered        23
fumble_offense_recovered        23
autoevent_passinterrupted       16
autoevent_passforward            9
Name: event, dtype: int64

# Developing a Ball Location Metric

In [148]:
plays_merge = plays[['gameId','playId','gamePlayId','absoluteYardlineNumber','ballCarrierId','quarter','possessionTeam','defensiveTeam','playResult']]

In [149]:
plays_merge

,gameId,playId,gamePlayId,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult
0,2022100908,3537,20221009083537,69,48723,4,ATL,TB,9
1,2022091103,3126,20220911033126,76,52457,4,PIT,CIN,3
2,2022091111,1148,20220911111148,40,42547,2,LV,LAC,15
3,2022100212,2007,20221002122007,47,46461,3,DEN,LV,7
4,2022091900,1372,20220919001372,75,47857,2,BUF,TEN,3
...,...,...,...,...,...,...,...,...,...
12481,2022100204,123,2022100204123,49,43293,1,DAL,WAS,1
12482,2022091200,3467,20220912003467,40,46189,4,SEA,DEN,2
12483,2022101605,3371,20221016053371,69,44860,4,CIN,NO,6
12484,2022100207,2777,20221002072777,44,52449,3,IND,TEN,-2


In [150]:
week1 = pd.merge(week1, plays_merge, on=['gameId','playId','gamePlayId'], how='left')

In [151]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult
0,2022090800,56,35472.0,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6
1,2022090800,56,35472.0,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6
2,2022090800,56,35472.0,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6
3,2022090800,56,35472.0,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6
4,2022090800,56,35472.0,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,NaN,football,49,football,56.220001,9.89,2.56,1.25,...,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9
1407435,2022091200,3826,NaN,football,50,football,56.060001,10.08,2.50,1.14,...,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9
1407436,2022091200,3826,NaN,football,51,football,55.889999,10.27,2.38,1.70,...,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9
1407437,2022091200,3826,NaN,football,52,football,55.730000,10.44,2.07,2.83,...,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9


In [152]:
games_merge = games[['gameId', 'week', 'homeTeamAbbr', 'visitorTeamAbbr']]

In [153]:
week1 = pd.merge(week1, games_merge, on=['gameId'], how='left')

In [154]:
print(week1.columns.tolist())

['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'club', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event', 'gamePlayId', 'tackler', 'assist', 'forcedFumble', 'pff_missedTackle', 'tackleDF', 'position', 'totalInches', 'mass', 'playType', 'absoluteYardlineNumber', 'ballCarrierId', 'quarter', 'possessionTeam', 'defensiveTeam', 'playResult', 'week', 'homeTeamAbbr', 'visitorTeamAbbr']


In [155]:
week1['nflId'] = week1['nflId'].fillna(0)

In [156]:
week1['nflId'] = week1['nflId'].astype(int)

In [157]:
week1['nflId'].isna().sum()

0

In [158]:
week1['xBallLocation'] = np.where(week1['ballCarrierId'] == week1['nflId'], week1['x'], 0)
week1['yBallLocation'] = np.where(week1['ballCarrierId'] == week1['nflId'], week1['y'], 0)

In [159]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,42489,1,BUF,LA,6,1,LA,BUF,0.0,0.0
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,42489,1,BUF,LA,6,1,LA,BUF,0.0,0.0
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,42489,1,BUF,LA,6,1,LA,BUF,0.0,0.0
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,42489,1,BUF,LA,6,1,LA,BUF,0.0,0.0
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,42489,1,BUF,LA,6,1,LA,BUF,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,...,53464,4,DEN,SEA,9,1,SEA,DEN,0.0,0.0
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,...,53464,4,DEN,SEA,9,1,SEA,DEN,0.0,0.0
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,...,53464,4,DEN,SEA,9,1,SEA,DEN,0.0,0.0
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,...,53464,4,DEN,SEA,9,1,SEA,DEN,0.0,0.0


To create the total yards on the play between the first frame and the last frame. 

In [160]:
week1['firstFrame'] = week1.groupby('gamePlayId')['frameId'].transform('min')

In [161]:
week1['lastFrame'] = week1.groupby('gamePlayId')['frameId'].transform('max')

In [162]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,BUF,LA,6,1,LA,BUF,0.0,0.0,1,22
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,BUF,LA,6,1,LA,BUF,0.0,0.0,1,22
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,BUF,LA,6,1,LA,BUF,0.0,0.0,1,22
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,BUF,LA,6,1,LA,BUF,0.0,0.0,1,22
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,BUF,LA,6,1,LA,BUF,0.0,0.0,1,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,...,DEN,SEA,9,1,SEA,DEN,0.0,0.0,1,53
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,...,DEN,SEA,9,1,SEA,DEN,0.0,0.0,1,53
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,...,DEN,SEA,9,1,SEA,DEN,0.0,0.0,1,53
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,...,DEN,SEA,9,1,SEA,DEN,0.0,0.0,1,53


In [163]:
week1['xFirstFrame'] = np.where(week1['frameId'] == week1['firstFrame'], week1['xBallLocation'], 0)
week1['xLastFrame'] = np.where(week1['frameId'] == week1['lastFrame'], week1['xBallLocation'], 0)

In [164]:
week1['xFirstFrame'] = week1.groupby('gamePlayId')['xFirstFrame'].transform(max)
week1['xLastFrame'] = week1.groupby('gamePlayId')['xLastFrame'].transform(max)

In [165]:
week1['xFrameChange'] = (week1['xLastFrame'] - week1['xFirstFrame']).abs()

In [166]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,...,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,...,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,...,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,...,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99


In [167]:
week1['frameId'].value_counts()

1      33879
2      33879
3      33879
4      33879
5      33879
       ...  
100       46
99        46
112       23
113       23
114       23
Name: frameId, Length: 114, dtype: int64

In [168]:
week1['xBallLocation'].isna().sum()

0

In [169]:
week1['yBallLocation'].isna().sum()

0

In [170]:
week1['ballCarrierId'].isna().sum()

0

In [171]:
week1['xBallLocation'].value_counts()

0.00      1346246
84.80          30
69.86          28
59.90          25
84.79          24
           ...   
100.23          1
95.17           1
104.40          1
105.05          1
60.83           1
Name: xBallLocation, Length: 10076, dtype: int64

In [172]:
week1['yBallLocation'].value_counts()

 0.00     1346248
 0.17         173
 29.80        103
 53.17         99
 23.67         98
           ...   
-1.82           1
-2.03           1
-3.47           1
 2.62           1
 7.41           1
Name: yBallLocation, Length: 5751, dtype: int64

Shows that there are 61,193 rows where the ball carrier ID matches the nfl ID. And below that it shows that there is a ball carrier ID on all the plays. So need to add the ball x and y to all the rows in the events dataset. 

In [173]:
len(week1.loc[week1.nflId == week1.ballCarrierId])

61193

In [174]:
plays['ballCarrierId'].isna().sum()

0

In [175]:
ball_location = week1.loc[week1.nflId == week1.ballCarrierId]

In [176]:
ball_location

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange
88,2022090800,56,42489,Stefon Diggs,1,BUF,80.60,38.29,6.23,0.90,...,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09
89,2022090800,56,42489,Stefon Diggs,2,BUF,80.48,37.68,6.24,1.27,...,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09
90,2022090800,56,42489,Stefon Diggs,3,BUF,80.34,37.09,5.98,2.48,...,1,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09
91,2022090800,56,42489,Stefon Diggs,4,BUF,80.19,36.53,5.67,3.37,...,1,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09
92,2022090800,56,42489,Stefon Diggs,5,BUF,80.01,36.00,5.43,3.65,...,1,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407275,2022091200,3826,53464,Javonte Williams,49,DEN,56.06,10.39,2.35,1.58,...,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99
1407276,2022091200,3826,53464,Javonte Williams,50,DEN,55.96,10.58,2.02,2.44,...,1,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99
1407277,2022091200,3826,53464,Javonte Williams,51,DEN,55.87,10.74,1.72,2.67,...,1,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99
1407278,2022091200,3826,53464,Javonte Williams,52,DEN,55.79,10.86,1.34,3.12,...,1,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99


In [177]:
ball_location = ball_location[['gamePlayId', 'frameId', 'xBallLocation', 'yBallLocation']]

In [178]:
ball_location

,gamePlayId,frameId,xBallLocation,yBallLocation
88,202209080056,1,80.60,38.29
89,202209080056,2,80.48,37.68
90,202209080056,3,80.34,37.09
91,202209080056,4,80.19,36.53
92,202209080056,5,80.01,36.00
...,...,...,...,...
1407275,20220912003826,49,56.06,10.39
1407276,20220912003826,50,55.96,10.58
1407277,20220912003826,51,55.87,10.74
1407278,20220912003826,52,55.79,10.86


In [179]:
ball_location['yBallLocation'].value_counts()

 0.17     173
 29.80    103
 53.17     99
 23.67     98
 29.79     90
         ... 
 15.10      1
-0.12       1
 47.39      1
 6.06       1
 7.41       1
Name: yBallLocation, Length: 5751, dtype: int64

In [180]:
week1 = pd.merge(week1, ball_location, on=['gamePlayId', 'frameId', 'xBallLocation', 'yBallLocation'], how='left')

In [181]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,1,LA,BUF,0.0,0.0,1,22,80.60,79.51,1.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,...,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,...,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,...,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,...,1,SEA,DEN,0.0,0.0,1,53,70.71,55.72,14.99


In [182]:
week1['xBallLocation'].value_counts()

0.00      1346246
84.80          30
69.86          28
59.90          25
84.79          24
           ...   
100.23          1
95.17           1
104.40          1
105.05          1
60.83           1
Name: xBallLocation, Length: 10076, dtype: int64

In [183]:
week1['yBallLocation'].value_counts()

 0.00     1346248
 0.17         173
 29.80        103
 53.17         99
 23.67         98
           ...   
-1.82           1
-2.03           1
-3.47           1
 2.62           1
 7.41           1
Name: yBallLocation, Length: 5751, dtype: int64

In [184]:
print(week1['xBallLocation'][week1['xBallLocation'] > 0].count())

61193


In [185]:
week1['xBallLocation'] = week1.groupby(['gamePlayId', 'frameId'])['xBallLocation'].transform(max)
week1['yBallLocation'] = week1.groupby(['gamePlayId', 'frameId'])['yBallLocation'].transform(max)

In [186]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,1,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,1,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,1,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,...,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,...,1,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,...,1,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,...,1,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99


In [187]:
week1['xBallLocation'].value_counts()

84.80     690
69.86     644
59.90     575
84.79     552
56.62     552
         ... 
19.70      23
14.77      23
93.34      23
109.14     23
60.83      23
Name: xBallLocation, Length: 10075, dtype: int64

In [188]:
week1['x'].value_counts()

76.12    250
75.24    248
86.26    246
86.29    243
79.79    241
        ... 
9.22       1
9.41       1
9.64       1
9.87       1
11.57      1
Name: x, Length: 21453, dtype: int64

In [189]:
week1['yBallLocation'].value_counts()

0.00     10028
0.17      3979
29.80     2369
53.17     2277
23.67     2254
         ...  
6.06        23
15.10       23
5.78        23
3.67        23
7.41        23
Name: yBallLocation, Length: 5540, dtype: int64

In [190]:
week1['y'].value_counts()

 29.750000    954
 23.750000    929
 23.770000    875
 29.830000    860
 23.810000    858
             ... 
 53.790001      1
 7.320000       1
 12.060000      1
-2.270000       1
 49.790001      1
Name: y, Length: 11519, dtype: int64

In [191]:
week1['ballDistance'] = np.sqrt((week1['xBallLocation'] - week1['x']) ** 2 + (week1['yBallLocation'] - week1['y']) ** 2)

In [192]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09,13.234145
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09,13.006721
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09,12.811690
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09,12.669021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,...,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,0.524976
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,...,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99,0.509902
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,...,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99,0.470425
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,...,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99,0.424265


In [193]:
week1['ballDistance'].value_counts()

0.000000    60760
0.610737       17
0.688840       17
0.546260       17
0.762955       17
            ...  
0.022362        1
0.022360        1
0.022360        1
0.014142        1
0.417852        1
Name: ballDistance, Length: 1115287, dtype: int64

In [194]:
week1['ballDistance'].describe()

count    1.407439e+06
mean     1.102222e+01
std      9.465209e+00
min      0.000000e+00
25%      4.004510e+00
50%      8.332401e+00
75%      1.624200e+01
max      7.795014e+01
Name: ballDistance, dtype: float64

In [195]:
week1['ballDistance'].max()

77.95014432315057

In [196]:
count = week1['ballDistance'].lt(1).sum()
print(count)

144883


In [197]:
count = week1['ballDistance'].lt(.05).sum()
print(count)

64905


In [198]:
count = week1['ballDistance'].lt(.0001).sum()
print(count)

60932


In [199]:
week1_tackle_bd = week1[week1.event == 'tackle']

In [200]:
week1_tackle_bd

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,...,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207
39,2022090800,56,38577,Bobby Wagner,18,LA,78.620000,32.31,1.76,3.13,...,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,1.695789
61,2022090800,56,41239,Aaron Donald,18,LA,91.040000,27.66,2.16,1.98,...,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,13.245384
83,2022090800,56,42392,Mitch Morse,18,BUF,87.200000,30.91,1.70,0.72,...,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,8.401220
105,2022090800,56,42489,Stefon Diggs,18,BUF,79.330000,33.85,0.62,1.31,...,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407222,2022091200,3826,52526,Albert Okwuegbunam,49,DEN,53.300000,14.66,3.14,3.23,...,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,5.084339
1407275,2022091200,3826,53464,Javonte Williams,49,DEN,56.060000,10.39,2.35,1.58,...,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,0.000000
1407328,2022091200,3826,54505,Boye Mafe,49,SEA,56.730000,7.82,0.98,2.21,...,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,2.655899
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.750000,6.92,1.88,2.49,...,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264


In [201]:
week1_tackle_bd['ballDistance'].mean()

10.527205866465636

In [202]:
week1_tackle_bd_tackler = week1_tackle_bd[week1_tackle_bd.tackler == 1]

In [203]:
week1_tackle_bd_tackler

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance
149,2022090800,56,43294,Jalen Ramsey,18,LA,78.66,33.87,1.10,3.17,...,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,0.670298
1161,2022090800,80,53532,Ernest Jones,26,LA,72.96,42.23,5.62,1.95,...,LA,BUF,73.10,42.57,1,30,83.80,72.33,11.47,0.367696
1436,2022090800,101,42816,Troy Hill,45,LA,69.08,39.50,1.80,4.97,...,LA,BUF,66.69,40.15,1,49,79.61,65.19,14.42,2.476812
2384,2022090800,122,38577,Bobby Wagner,29,LA,60.31,47.90,3.19,2.90,...,LA,BUF,59.89,47.51,1,33,68.96,59.08,9.88,0.573149
4541,2022090800,167,43298,Leonard Floyd,18,LA,46.84,25.08,4.16,6.63,...,LA,BUF,45.61,24.07,1,22,53.30,44.57,8.73,1.591540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402592,2022091200,3628,47871,Cody Barton,28,SEA,72.22,47.23,1.29,3.31,...,SEA,DEN,72.30,47.60,1,32,72.90,72.31,0.59,0.378550
1402938,2022091200,3685,35562,Al Woods,54,SEA,71.33,29.46,1.77,0.60,...,SEA,DEN,70.88,29.14,1,58,76.82,71.54,5.28,0.552178
1405238,2022091200,3723,52435,Jordyn Brooks,60,SEA,61.37,51.98,1.18,2.86,...,SEA,DEN,60.79,52.23,1,64,68.73,60.81,7.92,0.631585
1405985,2022091200,3747,47871,Cody Barton,19,SEA,68.00,40.59,2.45,5.22,...,SEA,DEN,68.41,41.26,1,23,64.16,69.21,5.05,0.785493


In [204]:
week1_tackle_bd_tackler['ballDistance'].mean()

1.1233829187167623

In [205]:
week1_tackle_bd_tackler['gamePlayId'].value_counts()

202209080056      1
20220911092579    1
2022091109743     1
2022091109767     1
2022091109792     1
                 ..
20220911041436    1
20220911041481    1
20220911041634    1
20220911041713    1
20220912003826    1
Name: gamePlayId, Length: 974, dtype: int64

In [206]:
week1_tackle_bd_assist = week1_tackle_bd[week1_tackle_bd.assist == 1]

In [207]:
week1_tackle_bd_assist

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance
3189,2022090800,146,38577,Bobby Wagner,52,LA,57.05,21.36,2.16,2.57,...,LA,BUF,57.06,22.45,1,56,64.16,57.04,7.12,1.090046
3637,2022090800,146,43335,A'Shawn Robinson,52,LA,57.74,21.23,1.08,2.23,...,LA,BUF,57.06,22.45,1,56,64.16,57.04,7.12,1.396710
5585,2022090800,191,47917,Greg Gaines,47,LA,43.65,41.92,4.67,4.00,...,LA,BUF,43.81,42.32,1,51,52.15,43.48,8.67,0.430813
14776,2022090800,593,47844,Taylor Rapp,29,LA,75.28,5.00,2.90,3.84,...,LA,BUF,73.71,5.83,1,33,86.54,72.66,13.88,1.775894
14908,2022090800,593,48026,Nick Scott,29,LA,76.09,6.69,0.51,3.94,...,LA,BUF,73.71,5.83,1,33,86.54,72.66,13.88,2.530613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398754,2022091200,3467,43387,Justin Simmons,21,DEN,41.94,18.59,3.20,3.21,...,SEA,DEN,42.04,19.08,1,25,40.29,42.16,1.87,0.500100
1399370,2022091200,3491,42393,Ronald Darby,32,DEN,44.31,1.20,5.76,5.56,...,SEA,DEN,43.36,1.11,1,36,38.00,44.70,6.70,0.954254
1399514,2022091200,3491,43387,Justin Simmons,32,DEN,43.20,1.47,6.97,3.69,...,SEA,DEN,43.36,1.11,1,36,38.00,44.70,6.70,0.393954
1403286,2022091200,3685,42827,Justin Coleman,54,SEA,70.84,28.54,1.80,2.44,...,SEA,DEN,70.88,29.14,1,58,76.82,71.54,5.28,0.601332


In [208]:
week1_tackle_bd_assist['ballDistance'].mean()

0.909999092776438

In [209]:
week1_tackle_bd_assist['gamePlayId'].value_counts()

2022090800146     2
2022091106466     2
2022091105294     2
2022091105403     2
2022091105445     2
                 ..
2022091107868     1
20220911072049    1
20220911072113    1
20220911072243    1
20220912003723    1
Name: gamePlayId, Length: 374, dtype: int64

In [210]:
count = week1_tackle_bd['ballDistance'].lt(1).sum()
print(count)

4209


There are 25,762 rows where the event is a tackle. The average ball distance for all players on those rows is 10.97 yards. There are 974 rows where the event is tackle and a player is identified as a tackler. The average ball distance 1.12 yards. There are 570 rows where the event is tackle and a player is identified as an assistor. The average ball distance is 0.91 yards. 

Overall, there are 3,158 rows where the event is a tackle and the ball distance is less than 1 yard. These could be possible offensive and defensive influencers. 

In [211]:
week1_tackle_bd_influencer = week1_tackle_bd[week1_tackle_bd.ballDistance <= 1.5]

In [212]:
week1_tackle_bd_influencer

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance
105,2022090800,56,42489,Stefon Diggs,18,BUF,79.330000,33.85,0.62,1.31,...,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,0.000000
149,2022090800,56,43294,Jalen Ramsey,18,LA,78.660000,33.87,1.10,3.17,...,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,0.670298
479,2022090800,56,53532,Ernest Jones,18,LA,78.250000,33.38,0.68,1.15,...,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,1.177837
861,2022090800,80,46076,Josh Allen,26,BUF,73.100000,42.57,5.28,5.20,...,LA,BUF,73.10,42.57,1,30,83.80,72.33,11.47,0.000000
1161,2022090800,80,53532,Ernest Jones,26,LA,72.960000,42.23,5.62,1.95,...,LA,BUF,73.10,42.57,1,30,83.80,72.33,11.47,0.367696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406215,2022091200,3747,0,football,19,football,68.680000,41.18,3.86,5.46,...,SEA,DEN,68.41,41.26,1,23,64.16,69.21,5.05,0.281603
1407010,2022091200,3826,52435,Jordyn Brooks,49,SEA,56.870000,9.65,2.34,2.30,...,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,1.097133
1407116,2022091200,3826,52456,Darrell Taylor,49,SEA,54.800000,10.72,2.39,2.55,...,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,1.302498
1407275,2022091200,3826,53464,Javonte Williams,49,DEN,56.060000,10.39,2.35,1.58,...,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,0.000000


In [213]:
week1_tackle_bd_influencer['ballDistance'].mean()

0.5400067576593087

In [214]:
week1_tackle_bd_influencer = week1_tackle_bd_influencer[(week1_tackle_bd_influencer.tackler == 0) & (week1_tackle_bd_influencer.assist == 0)]

In [215]:
week1_tackle_bd_influencer

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance
28835,2022090800,1385,40107,Micah Hyde,72,BUF,17.84,16.06,1.06,1.15,...,LA,BUF,17.06,17.25,1,76,42.41,16.61,25.80,1.422849
333371,2022091102,2556,47785,Nick Bosa,70,SF,41.98,32.75,0.04,0.04,...,CHI,SF,41.58,32.07,1,74,39.02,41.36,2.34,0.788923
339558,2022091102,2738,54504,Kyler Gordon,91,CHI,88.14,12.68,0.57,1.43,...,CHI,SF,89.09,11.95,1,95,91.27,89.31,1.96,1.198082
365580,2022091102,3859,46077,Roquan Smith,42,CHI,56.87,11.24,2.07,2.59,...,CHI,SF,56.25,10.98,1,46,49.28,56.26,6.98,0.672309
437867,2022091103,3126,47855,Germaine Pratt,55,CIN,73.80,51.88,5.45,3.40,...,CIN,PIT,72.66,51.82,1,59,78.22,72.16,6.06,1.141578
487703,2022091104,1020,53542,Derrick Barnes,61,DET,11.06,42.89,2.22,3.81,...,DET,PHI,11.11,42.10,1,65,16.88,11.25,5.63,0.791581
566536,2022091105,101,54502,Jalen Pitre,74,HOU,77.58,36.25,2.55,1.82,...,HOU,IND,77.37,36.93,1,78,83.08,77.52,5.56,0.711688
575794,2022091105,403,43497,Kamu Grugier-Hill,70,HOU,43.86,31.48,1.43,1.07,...,HOU,IND,43.39,31.31,1,74,52.72,43.82,8.90,0.499800
616657,2022091105,2132,46669,Jonathan Owens,70,HOU,70.22,23.77,2.12,1.93,...,HOU,IND,70.22,23.28,1,74,84.33,69.98,14.35,0.490000
617323,2022091105,2132,54502,Jalen Pitre,70,HOU,70.33,22.79,2.14,2.32,...,HOU,IND,70.22,23.28,1,74,84.33,69.98,14.35,0.502195


For all the rows where there is a tackle event and the ball distance is less than 1.5 yards, there are 4279 rows. When the tacklers and assistors are taken out, there are only 14 rows. That means there are only 14 potential influencers. 

In [216]:
#week1 = week1[week1.displayName != 'football']

# Setting Up A Player Direction vs Ball Carrier Direction

In [217]:
week1['dirBallCarrier'] = np.where(week1['ballCarrierId'] == week1['nflId'], week1['dir'], 0)

In [218]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,...,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,...,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,0.0
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,...,BUF,80.48,37.68,1,22,80.60,79.51,1.09,13.234145,0.0
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,...,BUF,80.34,37.09,1,22,80.60,79.51,1.09,13.006721,0.0
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,...,BUF,80.19,36.53,1,22,80.60,79.51,1.09,12.811690,0.0
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,...,BUF,80.01,36.00,1,22,80.60,79.51,1.09,12.669021,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,...,DEN,56.06,10.39,1,53,70.71,55.72,14.99,0.524976,0.0
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,...,DEN,55.96,10.58,1,53,70.71,55.72,14.99,0.509902,0.0
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,...,DEN,55.87,10.74,1,53,70.71,55.72,14.99,0.470425,0.0
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,...,DEN,55.79,10.86,1,53,70.71,55.72,14.99,0.424265,0.0


In [219]:
week1['dirBallCarrier'].value_counts()

0.00      1346247
339.49          9
194.72          9
32.99           9
206.16          9
           ...   
249.65          1
252.88          1
253.84          1
174.39          1
332.62          1
Name: dirBallCarrier, Length: 29324, dtype: int64

In [220]:
week1['dirBallCarrier'] = week1.groupby(['gamePlayId', 'frameId'])['dirBallCarrier'].transform(max)

In [221]:
week1['dirBallVariance'] = np.minimum(((week1['dir'] - week1['dirBallCarrier']).abs()), (360-(week1['dir'] - week1['dirBallCarrier']).abs()))

In [222]:
pd.options.display.max_columns = None
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09,13.234145,191.08,42.55
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09,13.006721,194.40,47.35
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09,12.811690,196.97,51.55
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09,12.669021,199.54,57.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,0.524976,332.62,NaN
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99,0.509902,331.45,NaN
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99,0.470425,330.51,NaN
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99,0.424265,326.53,NaN


In [223]:
week1['momentum'] = (week1['s'] * week1['mass']).round()
week1['force'] = (week1['a'] * week1['mass']).round()

In [224]:
week1['momemtumVector'] = (week1['momentum'] * week1['dirBallVariance']).round()
week1['forceVector'] = (week1['force'] * week1['dirBallVariance']).round()

# Create a Snap Event at FrameId 1

In [225]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09,13.234145,191.08,42.55,3483.0,1272.0,148202.0,54124.0
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09,13.006721,194.40,47.35,3274.0,1022.0,155024.0,48392.0
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09,12.811690,196.97,51.55,3003.0,1856.0,154805.0,95677.0
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09,12.669021,199.54,57.59,2690.0,2586.0,154917.0,148928.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,0.524976,332.62,NaN,NaN,NaN,NaN,NaN
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99,0.509902,331.45,NaN,NaN,NaN,NaN,NaN
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99,0.470425,330.51,NaN,NaN,NaN,NaN,NaN
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99,0.424265,326.53,NaN,NaN,NaN,NaN,NaN


In [226]:
week1 = week1[week1.event !='ball_snap']

In [227]:
week1['event'].value_counts()

first_contact                28773
tackle                       26928
pass_outcome_caught          15870
handoff                      15364
pass_arrived                 13915
out_of_bounds                 5037
run                           2737
man_in_motion                 1288
play_action                   1035
touchdown                     1012
fumble                         621
shift                          368
qb_slide                       350
pass_forward                   248
snap_direct                     46
line_set                        46
lateral                         45
autoevent_ballsnap              30
run_pass_option                 23
qb_sack                         23
pass_shovel                     23
fumble_defense_recovered        23
fumble_offense_recovered        23
autoevent_passinterrupted       16
autoevent_passforward            9
Name: event, dtype: int64

In [228]:
week1['event'] = np.where(week1['frameId']==1, 'snap', week1['event'])

In [229]:
week1['event'].value_counts()

snap                         33879
first_contact                28773
tackle                       26928
pass_outcome_caught          15870
handoff                      15364
pass_arrived                 12328
out_of_bounds                 5037
run                           2737
play_action                   1035
touchdown                     1012
man_in_motion                  989
fumble                         621
qb_slide                       350
shift                          322
pass_forward                   135
snap_direct                     46
lateral                         45
autoevent_ballsnap              30
line_set                        23
run_pass_option                 23
qb_sack                         23
fumble_defense_recovered        23
fumble_offense_recovered        23
autoevent_passinterrupted       16
autoevent_passforward            7
Name: event, dtype: int64

# Creating Yards from Different Events on Different Rows

In [230]:
week1

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09,13.234145,191.08,42.55,3483.0,1272.0,148202.0,54124.0
2,2022090800,56,35472,Rodger Saffold,3,BUF,88.560000,27.01,1.57,0.49,0.15,230.98,147.05,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.34,37.09,1,22,80.60,79.51,1.09,13.006721,194.40,47.35,3274.0,1022.0,155024.0,48392.0
3,2022090800,56,35472,Rodger Saffold,4,BUF,88.640000,26.90,1.44,0.89,0.14,232.38,145.42,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.19,36.53,1,22,80.60,79.51,1.09,12.811690,196.97,51.55,3003.0,1856.0,154805.0,95677.0
4,2022090800,56,35472,Rodger Saffold,5,BUF,88.720000,26.80,1.29,1.24,0.13,233.36,141.95,NaN,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.01,36.00,1,22,80.60,79.51,1.09,12.669021,199.54,57.59,2690.0,2586.0,154917.0,148928.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407434,2022091200,3826,0,football,49,football,56.220001,9.89,2.56,1.25,0.25,NaN,NaN,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,0.524976,332.62,NaN,NaN,NaN,NaN,NaN
1407435,2022091200,3826,0,football,50,football,56.060001,10.08,2.50,1.14,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,55.96,10.58,1,53,70.71,55.72,14.99,0.509902,331.45,NaN,NaN,NaN,NaN,NaN
1407436,2022091200,3826,0,football,51,football,55.889999,10.27,2.38,1.70,0.25,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,55.87,10.74,1,53,70.71,55.72,14.99,0.470425,330.51,NaN,NaN,NaN,NaN,NaN
1407437,2022091200,3826,0,football,52,football,55.730000,10.44,2.07,2.83,0.24,NaN,NaN,NaN,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,55.79,10.86,1,53,70.71,55.72,14.99,0.424265,326.53,NaN,NaN,NaN,NaN,NaN


In [231]:
week1['event'].value_counts()

snap                         33879
first_contact                28773
tackle                       26928
pass_outcome_caught          15870
handoff                      15364
pass_arrived                 12328
out_of_bounds                 5037
run                           2737
play_action                   1035
touchdown                     1012
man_in_motion                  989
fumble                         621
qb_slide                       350
shift                          322
pass_forward                   135
snap_direct                     46
lateral                         45
autoevent_ballsnap              30
line_set                        23
run_pass_option                 23
qb_sack                         23
fumble_defense_recovered        23
fumble_offense_recovered        23
autoevent_passinterrupted       16
autoevent_passforward            7
Name: event, dtype: int64

In [232]:
week1['event'] = np.where(week1['event']=='pass_outcome_caught', 'transfer', week1['event'])

In [233]:
week1['event'] = np.where(week1['event']=='handoff', 'transfer', week1['event'])

In [234]:
week1['event'].value_counts()

snap                         33879
transfer                     31234
first_contact                28773
tackle                       26928
pass_arrived                 12328
out_of_bounds                 5037
run                           2737
play_action                   1035
touchdown                     1012
man_in_motion                  989
fumble                         621
qb_slide                       350
shift                          322
pass_forward                   135
snap_direct                     46
lateral                         45
autoevent_ballsnap              30
line_set                        23
run_pass_option                 23
qb_sack                         23
fumble_defense_recovered        23
fumble_offense_recovered        23
autoevent_passinterrupted       16
autoevent_passforward            7
Name: event, dtype: int64

In [235]:
week1_events = week1[week1['event'].notna()]

In [236]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0
1,2022090800,56,35472,Rodger Saffold,2,BUF,88.470000,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.48,37.68,1,22,80.60,79.51,1.09,13.234145,191.08,42.55,3483.0,1272.0,148202.0,54124.0
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.800000,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0
22,2022090800,56,38577,Bobby Wagner,1,LA,78.250000,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407386,2022091200,3826,0,football,1,football,71.400002,17.91,20.33,3.93,2.05,NaN,NaN,snap,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,4.068931,178.63,NaN,NaN,NaN,NaN,NaN
1407389,2022091200,3826,0,football,4,football,70.860001,13.25,5.55,0.46,1.34,NaN,NaN,pass_arrived,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.69,12.73,1,53,70.71,55.72,14.99,0.547083,183.67,NaN,NaN,NaN,NaN,NaN
1407391,2022091200,3826,0,football,6,football,70.529999,12.14,5.55,1.14,0.56,NaN,NaN,transfer,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.57,11.96,1,53,70.71,55.72,14.99,0.184391,192.49,NaN,NaN,NaN,NaN,NaN
1407409,2022091200,3826,0,football,24,football,64.459999,5.05,5.21,4.07,0.51,NaN,NaN,first_contact,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,64.41,5.42,1,53,70.71,55.72,14.99,0.373363,260.10,NaN,NaN,NaN,NaN,NaN


In [237]:
print(week1_events.columns.tolist())

['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'club', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event', 'gamePlayId', 'tackler', 'assist', 'forcedFumble', 'pff_missedTackle', 'tackleDF', 'position', 'totalInches', 'mass', 'playType', 'absoluteYardlineNumber', 'ballCarrierId', 'quarter', 'possessionTeam', 'defensiveTeam', 'playResult', 'week', 'homeTeamAbbr', 'visitorTeamAbbr', 'xBallLocation', 'yBallLocation', 'firstFrame', 'lastFrame', 'xFirstFrame', 'xLastFrame', 'xFrameChange', 'ballDistance', 'dirBallCarrier', 'dirBallVariance', 'momentum', 'force', 'momemtumVector', 'forceVector']


In [238]:
week1_events.rename(columns={'tackle':'tackler'}, inplace=True)

In [239]:
week1_events['event'].value_counts()

snap                         33879
transfer                     31234
first_contact                28773
tackle                       26928
pass_arrived                 12328
out_of_bounds                 5037
run                           2737
play_action                   1035
touchdown                     1012
man_in_motion                  989
fumble                         621
qb_slide                       350
shift                          322
pass_forward                   135
snap_direct                     46
lateral                         45
autoevent_ballsnap              30
line_set                        23
run_pass_option                 23
qb_sack                         23
fumble_defense_recovered        23
fumble_offense_recovered        23
autoevent_passinterrupted       16
autoevent_passforward            7
Name: event, dtype: int64

In [240]:
drop_events = ['pass_arrived', 'run', 'man_in_motion', 'play_action', 'shift', 'qb_slide', 'pass_forward', 'touchdown', 
               'lateral', 'line_set', 'snap_direct', 'autoevent_ballsnap', 'fumble_offense_recovered', 'run_pass_option', 
               'pass_shovel', 'fumble_defense_recovered', 'qb_sack', 'autoevent_passinterrupted', 'autoevent_passforward']

In [241]:
week1_events = week1_events[~week1_events.event.isin(drop_events)]

In [242]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.800000,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0
22,2022090800,56,38577,Bobby Wagner,1,LA,78.250000,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0
27,2022090800,56,38577,Bobby Wagner,6,LA,78.110000,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.750000,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45,2441.0,3233.0,223229.0,295658.0
1407386,2022091200,3826,0,football,1,football,71.400002,17.91,20.33,3.93,2.05,NaN,NaN,snap,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,4.068931,178.63,NaN,NaN,NaN,NaN,NaN
1407391,2022091200,3826,0,football,6,football,70.529999,12.14,5.55,1.14,0.56,NaN,NaN,transfer,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.57,11.96,1,53,70.71,55.72,14.99,0.184391,192.49,NaN,NaN,NaN,NaN,NaN
1407409,2022091200,3826,0,football,24,football,64.459999,5.05,5.21,4.07,0.51,NaN,NaN,first_contact,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,64.41,5.42,1,53,70.71,55.72,14.99,0.373363,260.10,NaN,NaN,NaN,NaN,NaN


In [243]:
week1_events['event'].value_counts()

snap             33879
transfer         31234
first_contact    28773
tackle           26928
out_of_bounds     5037
fumble             621
Name: event, dtype: int64

In [244]:
week1_events_merge = week1_events[['gamePlayId', 'event']]

In [245]:
week1_events_merge

,gamePlayId,event
0,202209080056,snap
5,202209080056,transfer
17,202209080056,tackle
22,202209080056,snap
27,202209080056,transfer
...,...,...
1407381,20220912003826,tackle
1407386,20220912003826,snap
1407391,20220912003826,transfer
1407409,20220912003826,first_contact


In [246]:
event_dummies = pd.get_dummies(week1_events_merge['event'])

In [247]:
event_dummies

,first_contact,fumble,out_of_bounds,snap,tackle,transfer
0,0,0,0,1,0,0
5,0,0,0,0,0,1
17,0,0,0,0,1,0
22,0,0,0,1,0,0
27,0,0,0,0,0,1
...,...,...,...,...,...,...
1407381,0,0,0,0,1,0
1407386,0,0,0,1,0,0
1407391,0,0,0,0,0,1
1407409,1,0,0,0,0,0


In [248]:
week1_events_merge = pd.concat([week1_events_merge, event_dummies], axis=1) 
week1_events_merge.drop(['event'], inplace=True, axis=1)
week1_events_merge.drop(['gamePlayId'], inplace=True, axis=1)

In [249]:
week1_events_merge

,first_contact,fumble,out_of_bounds,snap,tackle,transfer
0,0,0,0,1,0,0
5,0,0,0,0,0,1
17,0,0,0,0,1,0
22,0,0,0,1,0,0
27,0,0,0,0,0,1
...,...,...,...,...,...,...
1407381,0,0,0,0,1,0
1407386,0,0,0,1,0,0
1407391,0,0,0,0,0,1
1407409,1,0,0,0,0,0


In [250]:
week1_events_merge.apply(pd.Series.value_counts)

,first_contact,fumble,out_of_bounds,snap,tackle,transfer
0,97699,125851,121435,92593,99544,95238
1,28773,621,5037,33879,26928,31234


In [251]:
week1_events = pd.concat([week1_events, week1_events_merge], axis=1)

In [252]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.800000,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0
22,2022090800,56,38577,Bobby Wagner,1,LA,78.250000,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0
27,2022090800,56,38577,Bobby Wagner,6,LA,78.110000,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.750000,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45,2441.0,3233.0,223229.0,295658.0,0,0,0,0,1,0
1407386,2022091200,3826,0,football,1,football,71.400002,17.91,20.33,3.93,2.05,NaN,NaN,snap,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,4.068931,178.63,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0
1407391,2022091200,3826,0,football,6,football,70.529999,12.14,5.55,1.14,0.56,NaN,NaN,transfer,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.57,11.96,1,53,70.71,55.72,14.99,0.184391,192.49,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,1
1407409,2022091200,3826,0,football,24,football,64.459999,5.05,5.21,4.07,0.51,NaN,NaN,first_contact,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,64.41,5.42,1,53,70.71,55.72,14.99,0.373363,260.10,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0


In [253]:
week1_events['playType'].value_counts()

run       60076
pass      59294
direct     7102
Name: playType, dtype: int64

In [254]:
week1['playType'].value_counts()

run       822602
pass      444061
direct    124361
Name: playType, dtype: int64

In [255]:
print(week1_events.columns.tolist())

['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'club', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event', 'gamePlayId', 'tackler', 'assist', 'forcedFumble', 'pff_missedTackle', 'tackleDF', 'position', 'totalInches', 'mass', 'playType', 'absoluteYardlineNumber', 'ballCarrierId', 'quarter', 'possessionTeam', 'defensiveTeam', 'playResult', 'week', 'homeTeamAbbr', 'visitorTeamAbbr', 'xBallLocation', 'yBallLocation', 'firstFrame', 'lastFrame', 'xFirstFrame', 'xLastFrame', 'xFrameChange', 'ballDistance', 'dirBallCarrier', 'dirBallVariance', 'momentum', 'force', 'momemtumVector', 'forceVector', 'first_contact', 'fumble', 'out_of_bounds', 'snap', 'tackle', 'transfer']


In [256]:
week1_events['xSnap'] = np.where(week1_events['snap'] == 1, week1_events['x'], 0)
week1_events['ySnap'] = np.where(week1_events['snap'] == 1, week1_events['y'], 0)
week1_events['xContact'] = np.where(week1_events['first_contact'] == 1, week1_events['x'], 0)
week1_events['yContact'] = np.where(week1_events['first_contact'] == 1, week1_events['y'], 0)
week1_events['xTransfer'] = np.where(week1_events['transfer'] == 1, week1_events['x'], 0)
week1_events['yTransfer'] = np.where(week1_events['transfer'] == 1, week1_events['y'], 0)
week1_events['xTackle'] = np.where(week1_events['tackle'] == 1, week1_events['x'], 0)
week1_events['yTackle'] = np.where(week1_events['tackle'] == 1, week1_events['y'], 0)

In [257]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.370000,27.27,0.000000,0.00,0.000000,0.00,0.000000,0.00
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.800000,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,0.000000,0.00,0.000000,0.00,88.800000,26.70,0.000000,0.00
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,0.000000,0.00,0.000000,0.00,0.000000,0.00,88.230000,27.09
22,2022090800,56,38577,Bobby Wagner,1,LA,78.250000,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.250000,27.53,0.000000,0.00,0.000000,0.00,0.000000,0.00
27,2022090800,56,38577,Bobby Wagner,6,LA,78.110000,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,0.000000,0.00,0.000000,0.00,78.110000,28.74,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.750000,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45,2441.0,3233.0,223229.0,295658.0,0,0,0,0,1,0,0.000000,0.00,0.000000,0.00,0.000000,0.00,50.750000,6.92
1407386,2022091200,3826,0,football,1,football,71.400002,17.91,20.33,3.93,2.05,NaN,NaN,snap,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,4.068931,178.63,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,71.400002,17.91,0.000000,0.00,0.000000,0.00,0.000000,0.00
1407391,2022091200,3826,0,football,6,football,70.529999,12.14,5.55,1.14,0.56,NaN,NaN,transfer,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.57,11.96,1,53,70.71,55.72,14.99,0.184391,192.49,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,1,0.000000,0.00,0.000000,0.00,70.529999,12.14,0.000000,0.00
1407409,2022091200,3826,0,football,24,football,64.459999,5.05,5.21,4.07,0.51,NaN,NaN,first_contact,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,64.41,5.42,1,53,70.71,55.72,14.99,0.373363,260.10,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0.000000,0.00,64.459999,5.05,0.000000,0.00,0.000000,0.00


To mark each event location for each player for all snaps. 

In [258]:
week1_events['xTransfer'].value_counts()

0.000000      95238
80.320000        14
74.810000        13
83.170000        11
77.700000        11
              ...  
68.809998         1
100.010000        1
102.620000        1
80.519997         1
70.529999         1
Name: xTransfer, Length: 10538, dtype: int64

In [259]:
event_locations = ['xSnap', 'ySnap', 'xContact', 'yContact', 'xTransfer', 'yTransfer', 'xTackle', 'yTackle']

In [260]:
for i in event_locations:
    week1_events[i] = week1_events.groupby(['gamePlayId', 'nflId'])[i].transform(max)

In [261]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.800000,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09
22,2022090800,56,38577,Bobby Wagner,1,LA,78.250000,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31
27,2022090800,56,38577,Bobby Wagner,6,LA,78.110000,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.750000,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45,2441.0,3233.0,223229.0,295658.0,0,0,0,0,1,0,53.280000,15.53,48.190000,13.00,51.220000,16.13,50.750000,6.92
1407386,2022091200,3826,0,football,1,football,71.400002,17.91,20.33,3.93,2.05,NaN,NaN,snap,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,4.068931,178.63,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,71.400002,17.91,64.459999,5.05,70.529999,12.14,56.220001,9.89
1407391,2022091200,3826,0,football,6,football,70.529999,12.14,5.55,1.14,0.56,NaN,NaN,transfer,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.57,11.96,1,53,70.71,55.72,14.99,0.184391,192.49,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,1,71.400002,17.91,64.459999,5.05,70.529999,12.14,56.220001,9.89
1407409,2022091200,3826,0,football,24,football,64.459999,5.05,5.21,4.07,0.51,NaN,NaN,first_contact,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,64.41,5.42,1,53,70.71,55.72,14.99,0.373363,260.10,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,71.400002,17.91,64.459999,5.05,70.529999,12.14,56.220001,9.89


In [262]:
week1_events['snapToTransfer'] = np.where((week1_events['xSnap'] > 0) & (week1_events['xTransfer'] > 0),
                                       (np.sqrt((week1_events['xTransfer'] - week1_events['xSnap']) ** 2 + (week1_events['yTransfer'] - week1_events['ySnap']) ** 2)), 0)

week1_events['transferToContact'] = np.where((week1_events['xTransfer'] > 0) & (week1_events['xContact'] > 0),
                                       (np.sqrt((week1_events['xContact'] - week1_events['xTransfer']) ** 2 + (week1_events['yContact'] - week1_events['yTransfer']) ** 2)), 0)

week1_events['transferToTackle'] = np.where((week1_events['xTransfer'] > 0) & (week1_events['xTackle'] > 0),
                                       (np.sqrt((week1_events['xTackle'] - week1_events['xTransfer']) ** 2 + (week1_events['yTackle'] - week1_events['yTransfer']) ** 2)), 0)

week1_events['contactToTackle'] = np.where((week1_events['xContact'] > 0) & (week1_events['xTackle'] > 0),
                                       (np.sqrt((week1_events['xTackle'] - week1_events['xContact']) ** 2 + (week1_events['yTackle'] - week1_events['yContact']) ** 2)), 0)

week1_events['snapToTackle'] = np.where((week1_events['xSnap'] > 0) & (week1_events['xTackle'] > 0),
                                       (np.sqrt((week1_events['xTackle'] - week1_events['xSnap']) ** 2 + (week1_events['yTackle'] - week1_events['ySnap']) ** 2)), 0)

week1_events['snapToContact'] = np.where((week1_events['xSnap'] > 0) & (week1_events['xContact'] > 0),
                                       (np.sqrt((week1_events['xContact'] - week1_events['xSnap']) ** 2 + (week1_events['yContact'] - week1_events['ySnap']) ** 2)), 0)
                                     

In [263]:
week1_events['transferToTackle'] = np.where((week1_events['xTransfer'] == 0) & (week1_events['xTackle'] > 0),
                                       (np.sqrt((week1_events['xTackle'] - week1_events['xSnap']) ** 2 + (week1_events['yTackle'] - week1_events['ySnap']) ** 2)),
                                           week1_events['transferToTackle'])

In [264]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle,snapToTransfer,transferToContact,transferToTackle,contactToTackle,snapToTackle,snapToContact
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.800000,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000
22,2022090800,56,38577,Bobby Wagner,1,LA,78.250000,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000
27,2022090800,56,38577,Bobby Wagner,6,LA,78.110000,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.750000,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45,2441.0,3233.0,223229.0,295658.0,0,0,0,0,1,0,53.280000,15.53,48.190000,13.00,51.220000,16.13,50.750000,6.92,2.145600,4.356352,9.221985,6.596969,8.974018,5.684101
1407386,2022091200,3826,0,football,1,football,71.400002,17.91,20.33,3.93,2.05,NaN,NaN,snap,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,4.068931,178.63,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,71.400002,17.91,64.459999,5.05,70.529999,12.14,56.220001,9.89,5.835221,9.333434,14.485804,9.556316,17.168366,14.613118
1407391,2022091200,3826,0,football,6,football,70.529999,12.14,5.55,1.14,0.56,NaN,NaN,transfer,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.57,11.96,1,53,70.71,55.72,14.99,0.184391,192.49,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,1,71.400002,17.91,64.459999,5.05,70.529999,12.1

In [265]:
print(week1_events.columns.tolist())

['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'club', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event', 'gamePlayId', 'tackler', 'assist', 'forcedFumble', 'pff_missedTackle', 'tackleDF', 'position', 'totalInches', 'mass', 'playType', 'absoluteYardlineNumber', 'ballCarrierId', 'quarter', 'possessionTeam', 'defensiveTeam', 'playResult', 'week', 'homeTeamAbbr', 'visitorTeamAbbr', 'xBallLocation', 'yBallLocation', 'firstFrame', 'lastFrame', 'xFirstFrame', 'xLastFrame', 'xFrameChange', 'ballDistance', 'dirBallCarrier', 'dirBallVariance', 'momentum', 'force', 'momemtumVector', 'forceVector', 'first_contact', 'fumble', 'out_of_bounds', 'snap', 'tackle', 'transfer', 'xSnap', 'ySnap', 'xContact', 'yContact', 'xTransfer', 'yTransfer', 'xTackle', 'yTackle', 'snapToTransfer', 'transferToContact', 'transferToTackle', 'contactToTackle', 'snapToTackle', 'snapToContact']


In [266]:
week1_events['snap'].value_counts()

0    92593
1    33879
Name: snap, dtype: int64

In [267]:
week1_events['snapToTackle'].value_counts()

0.000000     23124
1.609099        12
2.124759        12
0.223607        11
0.834865        11
             ...  
10.394619        2
3.234331         2
9.123185         2
12.693447        2
16.277899        2
Name: snapToTackle, Length: 26692, dtype: int64

In [268]:
week1_events['snapToTransfer'].value_counts()

0.000000    7102
0.156525      27
0.371080      23
0.610737      22
0.380789      20
            ... 
1.402355       2
1.941649       2
1.129690       2
1.414956       2
0.776660       2
Name: snapToTransfer, Length: 28439, dtype: int64

In [269]:
team_yards = week1_events.groupby('defensiveTeam').agg(
    SnapToTransferYards=('snapToTransfer', 'mean'),
    SnapToContactYards=('snapToContact', 'mean'),
    SnapToTackleYards=('snapToTackle', 'mean'),
    TransferToContactYards=('transferToContact', 'mean'),
    ContactToTackleYards=('contactToTackle', 'mean'),
    TransferToTackleYards=('transferToTackle', 'mean')
    
).reset_index()

# Sort teams by total tackles in descending order
team_yards_sorted = team_yards.sort_values(by='defensiveTeam')

team_yards_sorted.head(10)

,defensiveTeam,SnapToTransferYards,SnapToContactYards,SnapToTackleYards,TransferToContactYards,ContactToTackleYards,TransferToTackleYards
0,ARI,2.379863,6.075960,6.530552,4.394664,3.054507,5.569024
1,ATL,2.034675,5.053870,6.466549,2.548534,2.750456,5.198890
2,BAL,2.254037,5.677009,6.495663,3.949152,2.649573,5.387667
3,BUF,1.908629,4.726404,6.447677,3.080125,2.897483,5.417449
4,CAR,2.503904,6.495243,7.811297,4.398817,2.875919,6.398494
5,CHI,2.018989,6.845178,8.094697,4.227522,3.103257,7.190489
6,CIN,1.964738,4.357652,6.504927,3.017247,1.884004,5.421065
7,CLE,2.232308,6.160104,8.543512,4.527709,3.551721,7.378309
8,DAL,1.992932,5.347033,6.653140,4.051014,2.522937,5.689720
9,DEN,1.956242,5.142690,6.160258,2.999747,2.862508,5.334633


In [270]:
tackler_yards = week1_events.groupby('tackler').agg(
    SnapToTransferYards=('snapToTransfer', 'mean'),
    SnapToContactYards=('snapToContact', 'mean'),
    SnapToTackleYards=('snapToTackle', 'mean'),
    TransferToContactYards=('transferToContact', 'mean'),
    ContactToTackleYards=('contactToTackle', 'mean'),
    TransferToTackleYards=('transferToTackle', 'mean')
    
).reset_index()

# Sort teams by total tackles in descending order
tackler_yards_sorted = tackler_yards.sort_values(by='tackler')

tackler_yards_sorted

,tackler,SnapToTransferYards,SnapToContactYards,SnapToTackleYards,TransferToContactYards,ContactToTackleYards,TransferToTackleYards
0,0.0,2.083648,6.393825,8.322603,4.540181,3.755090,7.329454
1,1.0,2.256272,6.487223,7.568740,4.350821,2.849093,6.364186


In [271]:
ballDistance_yards = week1_events.groupby('tackler').agg(
    BallDistance=('ballDistance', 'mean'),

    
).reset_index()

# Sort teams by total tackles in descending order
ballDistance_yards_sorted = ballDistance_yards.sort_values(by='tackler')

ballDistance_yards_sorted

,tackler,BallDistance
0,0.0,6.218097
1,1.0,5.201252


In [272]:
ballDistanceAssist_yards = week1_events.groupby('assist').agg(
    BallDistance=('ballDistance', 'mean'),

    
).reset_index()

# Sort teams by total tackles in descending order
ballDistanceAssist_yards_sorted = ballDistanceAssist_yards.sort_values(by='assist')

ballDistanceAssist_yards_sorted

,assist,BallDistance
0,0.0,5.533681
1,1.0,5.840908


In [273]:
play_yards = week1_events.groupby('gamePlayId').agg(
    SnapToTransferYards=('snapToTransfer', 'mean'),
    SnapToContactYards=('snapToContact', 'mean'),
    SnapToTackleYards=('snapToTackle', 'mean'),
    TransferToContactYards=('transferToContact', 'mean'),
    ContactToTackleYards=('contactToTackle', 'mean'),
    TransferToTackleYards=('transferToTackle', 'mean')
    
).reset_index()

# Sort teams by total tackles in descending order
play_yards_sorted = play_yards.sort_values(by='gamePlayId')

play_yards_sorted.head(15)

,gamePlayId,SnapToTransferYards,SnapToContactYards,SnapToTackleYards,TransferToContactYards,ContactToTackleYards,TransferToTackleYards
0,202209080056,1.713325,0.000000,4.180924,0.000000,0.000000,2.832206
1,202209080080,0.000000,5.812244,8.015850,0.000000,2.544397,8.015850
2,202209110057,0.000000,15.833565,17.979163,0.000000,2.602940,17.979163
3,202209110078,3.523497,7.794187,8.518467,7.117346,5.016372,9.355119
4,202209110185,2.209501,3.209115,6.068971,1.052248,3.761058,4.548887
5,202209110286,2.250698,7.268099,11.388414,5.616280,4.909658,9.896973
6,202209110358,1.802867,2.853724,5.094555,1.104436,2.457144,3.509084
7,202209110382,2.286150,0.000000,0.000000,0.000000,0.000000,0.000000
8,202209110458,2.162537,3.790390,6.203213,1.705478,2.638906,4.283759
9,202209110486,2.446997,5.468533,33.384269,3.901983,31.726155,33.526996


In [274]:
play_yards = week1_events.groupby('gamePlayId').agg(
    SnapToTransferYards=('snapToTransfer', 'mean'),
    SnapToContactYards=('snapToContact', 'mean'),
    SnapToTackleYards=('snapToTackle', 'mean'),
    TransferToContactYards=('transferToContact', 'mean'),
    ContactToTackleYards=('contactToTackle', 'mean'),
    TransferToTackleYards=('transferToTackle', 'mean')
    
).reset_index()

# Sort teams by total tackles in descending order
play_yards_sorted = play_yards.sort_values(by='gamePlayId')

play_yards_sorted.head(15)

,gamePlayId,SnapToTransferYards,SnapToContactYards,SnapToTackleYards,TransferToContactYards,ContactToTackleYards,TransferToTackleYards
0,202209080056,1.713325,0.000000,4.180924,0.000000,0.000000,2.832206
1,202209080080,0.000000,5.812244,8.015850,0.000000,2.544397,8.015850
2,202209110057,0.000000,15.833565,17.979163,0.000000,2.602940,17.979163
3,202209110078,3.523497,7.794187,8.518467,7.117346,5.016372,9.355119
4,202209110185,2.209501,3.209115,6.068971,1.052248,3.761058,4.548887
5,202209110286,2.250698,7.268099,11.388414,5.616280,4.909658,9.896973
6,202209110358,1.802867,2.853724,5.094555,1.104436,2.457144,3.509084
7,202209110382,2.286150,0.000000,0.000000,0.000000,0.000000,0.000000
8,202209110458,2.162537,3.790390,6.203213,1.705478,2.638906,4.283759
9,202209110486,2.446997,5.468533,33.384269,3.901983,31.726155,33.526996


In [310]:
play_yards = week1_events.groupby('playType').agg(
    SnapToTransferYards=('snapToTransfer', 'mean'),
    TransferToTackleYards=('transferToTackle', 'mean')
    
).reset_index()

# Sort teams by total tackles in descending order
play_yards_sorted = play_yards.sort_values(by='playType')

play_yards_sorted.head(15)

,playType,SnapToTransferYards,TransferToTackleYards
0,direct,0.000000,9.575604
1,pass,1.822107,5.911435
2,run,2.442688,7.723708


In [275]:
table = pd.pivot_table(week1_events, values='transferToTackle', index=['defensiveTeam'],
                       columns=['quarter'], aggfunc="mean", margins="True", margins_name='Overall')
table = table.assign(sortkey=table.index == 'Overall')\
                .sort_values(['sortkey','Overall'], ascending=[False, True])\
                .drop('sortkey', axis=1)
table

quarter,1,2,3,4,5,Overall
defensiveTeam,,,,,,
Overall,6.242509,5.386248,6.028263,5.569597,4.408129,5.753953
MIA,3.688170,6.579876,4.481879,5.110214,NaN,4.636209
TB,6.300670,3.635419,6.369438,3.233279,NaN,4.981223
KC,8.336853,1.646904,5.702527,5.329835,NaN,4.998949
LAC,4.932710,4.376296,7.522525,2.811332,NaN,5.040287
IND,4.847583,5.659495,4.134897,8.405685,1.446990,5.076414
WAS,6.119265,5.030261,5.089870,3.954489,NaN,5.105746
PIT,6.603705,4.553308,6.134408,4.937236,2.992608,5.192909
ATL,5.219982,4.082639,7.243718,3.321552,NaN,5.198890


In [276]:
table = pd.pivot_table(week1_events, values='transferToTackle', index=['defensiveTeam'],
                       columns=['playType'], aggfunc="mean", margins="True", margins_name='Overall')
table = table.assign(sortkey=table.index == 'Overall')\
                .sort_values(['sortkey','Overall'], ascending=[False, True])\
                .drop('sortkey', axis=1)
table

playType,direct,pass,run,Overall
defensiveTeam,,,,
Overall,6.815641,4.398034,6.966713,5.753953
MIA,NaN,3.222079,6.130007,4.636209
TB,9.993963,3.049733,7.317401,4.981223
KC,6.582760,4.339133,5.561579,4.998949
LAC,NaN,4.310374,6.205042,5.040287
IND,1.834889,3.833175,6.215766,5.076414
WAS,2.742011,2.735881,8.299380,5.105746
PIT,8.400815,4.153009,5.895377,5.192909
ATL,6.025136,3.358026,7.362518,5.198890


In [277]:
#table = pd.pivot_table(df, values='D', index=['A', 'B'],
#                       columns=['C'], aggfunc="sum")

# To Track the Ball For Each Event for True Tackling Success

In [278]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle,snapToTransfer,transferToContact,transferToTackle,contactToTackle,snapToTackle,snapToContact
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.800000,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000
22,2022090800,56,38577,Bobby Wagner,1,LA,78.250000,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000
27,2022090800,56,38577,Bobby Wagner,6,LA,78.110000,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.750000,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45,2441.0,3233.0,223229.0,295658.0,0,0,0,0,1,0,53.280000,15.53,48.190000,13.00,51.220000,16.13,50.750000,6.92,2.145600,4.356352,9.221985,6.596969,8.974018,5.684101
1407386,2022091200,3826,0,football,1,football,71.400002,17.91,20.33,3.93,2.05,NaN,NaN,snap,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,4.068931,178.63,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,71.400002,17.91,64.459999,5.05,70.529999,12.14,56.220001,9.89,5.835221,9.333434,14.485804,9.556316,17.168366,14.613118
1407391,2022091200,3826,0,football,6,football,70.529999,12.14,5.55,1.14,0.56,NaN,NaN,transfer,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.57,11.96,1,53,70.71,55.72,14.99,0.184391,192.49,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,1,71.400002,17.91,64.459999,5.05,70.529999,12.1

In [279]:
week1_events['xBallLocation'].nunique()

4169

In [280]:
week1_events['xBallLocation'].value_counts()

54.95    161
45.16    138
84.22    115
81.65    115
70.63    115
        ... 
43.22     23
56.06     23
79.44     22
52.96     22
63.36     22
Name: xBallLocation, Length: 4169, dtype: int64

In [281]:
week1_events['gamePlayId'].nunique()

1473

In [282]:
week1_events['x'].nunique()

14793

In [283]:
week1_events['ballCarrierId'].nunique()

280

In [284]:
week1_events['transfer'].value_counts()

0    95238
1    31234
Name: transfer, dtype: int64

In [285]:
week1_events['xTransfer'].value_counts()

0.00      7102
80.32       56
74.81       50
86.10       45
52.12       44
          ... 
108.09       2
102.25       2
115.44       2
99.20        2
114.20       2
Name: xTransfer, Length: 10525, dtype: int64

In [286]:
week1_events['xBallSnap'] = np.where(week1_events['snap'] == 1, week1_events['xBallLocation'], 0)
week1_events['xBallTransfer'] = np.where(week1_events['transfer'] == 1, week1_events['xBallLocation'], 0)
week1_events['xBallContact'] = np.where(week1_events['first_contact'] == 1, week1_events['xBallLocation'], 0)
week1_events['xBallTackle'] = np.where(week1_events['tackle'] == 1, week1_events['xBallLocation'], 0)

In [287]:
pd.options.display.max_columns = None
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle,snapToTransfer,transferToContact,transferToTackle,contactToTackle,snapToTackle,snapToContact,xBallSnap,xBallTransfer,xBallContact,xBallTackle
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,0.00,0.00,0.00
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.800000,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,0.00,79.85,0.00,0.00
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,0.00,0.00,0.00,79.33
22,2022090800,56,38577,Bobby Wagner,1,LA,78.250000,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,80.60,0.00,0.00,0.00
27,2022090800,56,38577,Bobby Wagner,6,LA,78.110000,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,0.00,79.85,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.750000,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45,2441.0,3233.0,223229.0,295658.0,0,0,0,0,1,0,53.280000,15.53,48.190000,13.00,51.220000,16.13,50.750000,6.92,2.145600,4.356352,9.221985,6.596969,8.974018,5.684101,0.00,0.00,0.00,56.06
1407386,2022091200,3826,0,football,1,football,71.400002,17.91,20.33,3.93,2.05,NaN,NaN,snap,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,4.068931,178.63,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,71.400002,17.91,64.459999,5.05,70.529999,12.14,56.220001,9.89,5.835221,9.333434,14.485804,9.556316,17.168366,14.613118,70.71,0.00,0.00,0.00
1407391,2022091200,3826,0,football,6,football,70.529999,12.14,5.55,1.14,0.56,NaN,NaN,transfer

In [288]:
week1_events['tackler'].value_counts()

1.0    4566
0.0    3252
Name: tackler, dtype: int64

In [289]:
week1_events['playId'].nunique()

1247

In [290]:
week1_events['xBallSnap'].value_counts()

0.00     92593
79.59       69
84.33       69
64.16       69
69.14       69
         ...  
22.38       23
77.26       23
70.79       23
40.39       23
70.71       23
Name: xBallSnap, Length: 1384, dtype: int64

In [291]:
events_ball = ['xBallSnap', 'xBallTransfer', 'xBallContact', 'xBallTackle']

In [292]:
for i in events_ball:
    week1_events[i] = week1_events.groupby(['gamePlayId'])[i].transform(max)

In [293]:
week1_events = week1_events[week1_events['xBallTackle']!=0]

In [294]:
week1_events['xBallTransfer'] = np.where(week1_events['xBallTransfer']== 0, week1_events['xBallSnap'], week1_events['xBallTransfer'])
week1_events['xBallContact'] = np.where(week1_events['xBallContact']== 0, week1_events['xBallTackle'], week1_events['xBallContact'])

In [295]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle,snapToTransfer,transferToContact,transferToTackle,contactToTackle,snapToTackle,snapToContact,xBallSnap,xBallTransfer,xBallContact,xBallTackle
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.800000,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33
22,2022090800,56,38577,Bobby Wagner,1,LA,78.250000,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,80.60,79.85,79.33,79.33
27,2022090800,56,38577,Bobby Wagner,6,LA,78.110000,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,80.60,79.85,79.33,79.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.750000,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45,2441.0,3233.0,223229.0,295658.0,0,0,0,0,1,0,53.280000,15.53,48.190000,13.00,51.220000,16.13,50.750000,6.92,2.145600,4.356352,9.221985,6.596969,8.974018,5.684101,70.71,70.57,64.41,56.06
1407386,2022091200,3826,0,football,1,football,71.400002,17.91,20.33,3.93,2.05,NaN,NaN,snap,20220912003826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,70.71,13.90,1,53,70.71,55.72,14.99,4.068931,178.63,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,71.400002,17.91,64.459999,5.05,70.529999,12.14,56.220001,9.89,5.835221,9.333434,14.485804,9.556316,17.168366,14.613118,70.71,70.57,64.41,56.06
1407391,2022091200,3826,0,football,6,football,70.529999,12.14,5.55,1.14,

In [296]:
week1_events['xBallSnap'].value_counts()

64.16    253
79.59    230
85.33    184
85.38    184
13.72    184
        ... 
62.44     46
47.51     45
35.31     45
69.04     45
78.13     45
Name: xBallSnap, Length: 1113, dtype: int64

In [297]:
week1_events['xSnap'].value_counts()

86.260000    47
81.130000    46
84.230000    44
46.530000    44
50.760000    43
             ..
19.420000     2
43.189999     2
82.980003     1
48.189999     1
35.590000     1
Name: xSnap, Length: 9926, dtype: int64

In [298]:
week1_events['ballSnapToTransfer'] = (week1_events['xBallTransfer'] - week1_events['xBallSnap']).abs()
week1_events['ballSnapToContact'] = (week1_events['xBallContact'] - week1_events['xBallSnap']).abs()
week1_events['ballTransferToContact'] = (week1_events['xBallContact'] - week1_events['xBallTransfer']).abs()
week1_events['ballContactToTackle'] = (week1_events['xBallTackle'] - week1_events['xBallContact']).abs()
week1_events['ballSnapToTackle'] = (week1_events['xBallTackle'] - week1_events['xBallSnap']).abs()
week1_events['ballTransferToTackle'] = (week1_events['xBallTackle'] - week1_events['xBallTransfer']).abs()
week1_events['ballTackleToEnd'] = (week1_events['xLastFrame'] - week1_events['xBallTackle']).abs()

In [299]:
week1_events

,gameId,playId,nflId,displayName,frameId,club,x,y,s,a,dis,o,dir,event,gamePlayId,tackler,assist,forcedFumble,pff_missedTackle,tackleDF,position,totalInches,mass,playType,absoluteYardlineNumber,ballCarrierId,quarter,possessionTeam,defensiveTeam,playResult,week,homeTeamAbbr,visitorTeamAbbr,xBallLocation,yBallLocation,firstFrame,lastFrame,xFirstFrame,xLastFrame,xFrameChange,ballDistance,dirBallCarrier,dirBallVariance,momentum,force,momemtumVector,forceVector,first_contact,fumble,out_of_bounds,snap,tackle,transfer,xSnap,ySnap,xContact,yContact,xTransfer,yTransfer,xTackle,yTackle,snapToTransfer,transferToContact,transferToTackle,contactToTackle,snapToTackle,snapToContact,xBallSnap,xBallTransfer,xBallContact,xBallTackle,ballSnapToTransfer,ballSnapToContact,ballTransferToContact,ballContactToTackle,ballSnapToTackle,ballTransferToTackle,ballTackleToEnd
0,2022090800,56,35472,Rodger Saffold,1,BUF,88.370000,27.27,1.62,1.15,0.16,231.74,147.90,snap,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,13.483816,189.38,41.48,3378.0,2398.0,140119.0,99469.0,0,0,0,1,0,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
5,2022090800,56,35472,Rodger Saffold,6,BUF,88.800000,26.70,1.15,1.42,0.12,234.48,139.41,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,12.614856,202.20,62.79,2398.0,2961.0,150570.0,185921.0,0,0,0,0,0,1,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
17,2022090800,56,35472,Rodger Saffold,18,BUF,88.230000,27.09,1.72,0.26,0.18,288.76,302.08,tackle,202209080056,NaN,NaN,NaN,NaN,NaN,G,77.0,2085.42,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.33,33.85,1,22,80.60,79.51,1.09,11.176207,78.02,135.94,3587.0,542.0,487617.0,73679.0,0,0,0,0,1,0,88.370000,27.27,0.000000,0.00,88.800000,26.70,88.230000,27.09,0.714003,0.000000,0.690652,0.000000,0.228035,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
22,2022090800,56,38577,Bobby Wagner,1,LA,78.250000,27.53,1.24,3.81,0.11,4.20,331.67,snap,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,80.60,38.29,1,22,80.60,79.51,1.09,11.013632,189.38,142.29,1800.0,5532.0,256122.0,787148.0,0,0,0,1,0,0,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
27,2022090800,56,38577,Bobby Wagner,6,LA,78.110000,28.74,3.35,2.62,0.32,349.47,357.71,transfer,202209080056,NaN,NaN,NaN,NaN,NaN,ILB,72.0,1452.00,pass,85,42489,1,BUF,LA,6,1,LA,BUF,79.85,35.59,1,22,80.60,79.51,1.09,7.067538,202.20,155.51,4864.0,3804.0,756401.0,591560.0,0,0,0,0,0,1,78.250000,27.53,0.000000,0.00,78.110000,28.74,78.620000,32.31,1.218072,0.000000,3.606245,0.000000,4.794299,0.000000,80.60,79.85,79.33,79.33,0.75,1.27,0.52,0.00,1.27,0.52,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407381,2022091200,3826,54618,Tariq Woolen,49,SEA,50.750000,6.92,1.88,2.49,0.19,33.48,64.07,tackle,20220912003826,NaN,NaN,NaN,NaN,NaN,CB,76.0,1298.33,pass,65,53464,4,DEN,SEA,9,1,SEA,DEN,56.06,10.39,1,53,70.71,55.72,14.99,6.343264,332.62,91.45,2441.0,3233.0,223229.0,295658.0,0,0,0,0,1,0,53.280000,15.53,48.190000,13.00,51.220000,16.13,50.750000,6.92,2.145600,4.356352,9.221985,6.596969,8.974018,5.684101,70.71,70.57,64.41,56.06,0.14,6.30,6.16,8.35,14.65,14.51,0.34
1407386,2022091200,3826,0,football,1,football,71.400002,17.91,20.33,3.93,2.05,NaN,NaN,snap,20220912

In [300]:
week1_events['xBallSnap'].value_counts()

64.16    253
79.59    230
85.33    184
85.38    184
13.72    184
        ... 
62.44     46
47.51     45
35.31     45
69.04     45
78.13     45
Name: xBallSnap, Length: 1113, dtype: int64

In [301]:
week1_events['xBallTransfer'].value_counts()

42.64    276
45.54    276
70.63    230
98.09    184
97.40    184
        ... 
42.78     46
69.04     45
35.31     45
47.51     45
78.13     45
Name: xBallTransfer, Length: 1111, dtype: int64

In [302]:
week1_events['xBallContact'].value_counts()

83.25     184
73.11     184
84.80     184
53.11     184
101.47    184
         ... 
55.52      46
20.96      46
52.96      45
79.44      45
63.36      45
Name: xBallContact, Length: 1087, dtype: int64

In [303]:
week1_events['xBallTackle'].value_counts()

46.56    345
42.71    276
77.81    276
54.95    276
41.90    253
        ... 
20.96     46
83.50     45
63.36     45
79.44     45
52.96     45
Name: xBallTackle, Length: 1095, dtype: int64

In [304]:
week1_events['tackle'].value_counts()

0    76429
1    26928
Name: tackle, dtype: int64

In [305]:
week1_events['playResult']

0          6
5          6
17         6
22         6
27         6
          ..
1407381    9
1407386    9
1407391    9
1407409    9
1407434    9
Name: playResult, Length: 103357, dtype: int64

In [306]:
play_yards = week1_events.groupby('gamePlayId').agg(
    PlayResult=('playResult', max),
    FrameChange=('xFrameChange', max),
    BallSnapToTransfer=('ballSnapToTransfer', max),
    BallSnapToContact=('ballSnapToContact', max),
    BallSnapToTackle=('ballSnapToTackle', max),
    BallTransferToContact=('ballTransferToContact', max),
    BallContactToTackle=('ballContactToTackle', max),
    BallTransferToTackle=('ballTransferToTackle', max),
    BallTackletoEnd=('ballTackleToEnd', max)
    
).reset_index()

# Sort teams by total tackles in descending order
play_yards_sorted = play_yards.sort_values(by='gamePlayId')

play_yards_sorted.head(15)

,gamePlayId,PlayResult,FrameChange,BallSnapToTransfer,BallSnapToContact,BallSnapToTackle,BallTransferToContact,BallContactToTackle,BallTransferToTackle,BallTackletoEnd
0,202209080056,6,1.09,0.75,1.27,1.27,0.52,0.00,0.52,0.18
1,202209080080,7,11.47,0.00,8.27,10.70,8.27,2.43,10.70,0.77
2,202209110057,11,18.24,0.00,13.21,16.59,13.21,3.38,16.59,1.65
3,202209110078,6,13.94,0.78,10.89,13.61,10.11,2.72,12.83,0.33
4,202209110185,-5,3.08,0.14,0.30,3.11,0.44,2.81,3.25,0.03
5,202209110286,3,7.91,0.17,5.30,8.20,5.13,2.90,8.03,0.29
6,202209110358,3,4.94,1.34,2.30,4.51,0.96,2.21,3.17,0.43
7,202209110458,-1,6.08,1.59,3.15,5.29,1.56,2.14,3.70,0.79
8,202209110486,50,58.80,2.39,10.47,56.37,8.08,45.90,53.98,2.43
9,202209110556,4,9.02,0.12,4.61,8.76,4.49,4.15,8.64,0.26


In [307]:
team_yards = week1_events.groupby('defensiveTeam').agg(
    TransferToTackleYards=('transferToTackle', 'mean'),
    BallTransferToTackle=('ballTransferToTackle', 'mean'),
    ForceVector=('forceVector', 'mean')
    
).reset_index()

# Sort teams by total tackles in descending order
team_yards_sorted = team_yards.sort_values(by='defensiveTeam')

team_yards_sorted

,defensiveTeam,TransferToTackleYards,BallTransferToTackle,ForceVector
0,ARI,7.474217,8.237105,170621.181220
1,ATL,6.079347,6.595887,150321.428152
2,BAL,6.751213,5.571296,192062.682099
3,BUF,6.170869,5.851126,206531.547562
4,CAR,7.523504,7.123022,152357.507992
5,CHI,8.876535,8.647586,139566.692790
6,CIN,6.542665,6.417759,164223.936851
7,CLE,8.309552,8.987087,165887.688858
8,DAL,6.916914,7.158431,155951.767380
9,DEN,7.067249,7.205299,183247.054390
